In [2]:
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

from sklearn.preprocessing import OneHotEncoder, StandardScaler
OHE = OneHotEncoder(sparse=False)
pd.set_option('display.max_rows', 30)

In [3]:
# 月日如果用起来要one-hot，之后产生的feature就太多了而且感觉没必要，可能考虑is_special更合适
# 但是前后依赖这里怎么办呢？
# train[['month','day']] = train.date.str.split('-', expand=True)
def get_x_y(df):
    data = df.drop(columns=['Unnamed: 0', 'route', 'calculated_day', 'date'])
    ohe = OHE.fit_transform(data[['hr', 'weekday']])
    ohe_df = pd.DataFrame(ohe, columns= OHE.get_feature_names(input_features=['hr', 'weekday']))
    data = pd.concat([data, ohe_df],axis = 1)
    data = data.drop(columns=['hr', 'weekday'])
    x = data.drop(columns=['label'])
    y = data.label
    return x, y

In [4]:
def mape_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true))


def scoring(reg, x, y):
    pred = reg.predict(x)
    return -mape_error(pred, y)

## Read the Real Value at Each Time Window

In [5]:


import math
from datetime import datetime,timedelta
import numpy as np
import pandas as pd
names=locals()

link_time={}
time_train={}
time_predict={}
time_check={}
test_time_train={}
test_time_check={}
weathers={}
rainingTotalTime={}

file_path='/Users/vayne/Desktop/DM_Project_0522due/dataSet_phase2/table5.csv'
# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
    # Step 2: Create a dictionary to store travel time for each route per time window
    travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
    for i in range(len(traj_data)):
        each_traj = traj_data[i].replace('"', '').split(',')
        intersection_id = each_traj[0]
        tollgate_id = each_traj[1]

        route_id = intersection_id + '_' + tollgate_id
        if route_id not in travel_times.keys():
            travel_times[route_id] = {}

        trace_start_time = each_traj[3]
        travel_seq = each_traj[4]
        trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
        time_window_minute = math.floor(trace_start_time.minute / 20) * 20
        start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
        time=start_time_window
        tt = float(each_traj[-1])
        # 國慶不管，因道路狀態不一樣
        #if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        #    continue
        # 中秋節不管，因道路狀態不一樣
        if(time.month==9 and time.day in [15,16,17]):
            continue
        if start_time_window not in travel_times[route_id].keys():
            travel_times[route_id][start_time_window] = [tt]
        else:
            travel_times[route_id][start_time_window].append(tt)

In [6]:
real_value=[]
real_value_dict={}
#fw = open('/Users/vayne/Desktop/dm_pro_engin/real_value.csv', 'w')
#fw.writelines(','.join(['"intersection_id"', '"tollgate_id"', '"time_window"', '"avg_travel_time"']) + '\n')
for route in travel_times.keys():
    if route not in real_value_dict.keys():
        real_value_dict[route] = {}
    route_time_windows = list(travel_times[route].keys())
    route_time_windows.sort()
    for time_window_start in route_time_windows:
        time_window_end = time_window_start + timedelta(minutes=20)
        tt_set = travel_times[route][time_window_start]
        avg_tt = round(sum(tt_set) / float(len(tt_set)), 2)
        out_line =[str(route),time_window_start, str(avg_tt)]
        real_value.append(out_line)    
        
        

In [7]:
real_value

[['B_3', datetime.datetime(2016, 10, 18, 0, 0), '64.69'],
 ['B_3', datetime.datetime(2016, 10, 18, 0, 40), '75.37'],
 ['B_3', datetime.datetime(2016, 10, 18, 1, 40), '70.14'],
 ['B_3', datetime.datetime(2016, 10, 18, 2, 0), '96.03'],
 ['B_3', datetime.datetime(2016, 10, 18, 3, 40), '76.88'],
 ['B_3', datetime.datetime(2016, 10, 18, 6, 0), '54.75'],
 ['B_3', datetime.datetime(2016, 10, 18, 6, 20), '121.02'],
 ['B_3', datetime.datetime(2016, 10, 18, 6, 40), '94.04'],
 ['B_3', datetime.datetime(2016, 10, 18, 7, 0), '120.6'],
 ['B_3', datetime.datetime(2016, 10, 18, 7, 20), '73.4'],
 ['B_3', datetime.datetime(2016, 10, 18, 7, 40), '89.91'],
 ['B_3', datetime.datetime(2016, 10, 18, 8, 0), '126.4'],
 ['B_3', datetime.datetime(2016, 10, 18, 8, 20), '115.52'],
 ['B_3', datetime.datetime(2016, 10, 18, 8, 40), '98.86'],
 ['B_3', datetime.datetime(2016, 10, 18, 9, 0), '131.15'],
 ['B_3', datetime.datetime(2016, 10, 18, 9, 20), '151.95'],
 ['B_3', datetime.datetime(2016, 10, 18, 9, 40), '117.69'],

In [8]:
real_value_needed=[]
for i in real_value:
    if i[1].hour in [8,9,17,18]:
        real_value_needed.append(i)
        

In [9]:
#convert to dataframe
feature=['route','time','label']
real_value_df=pd.DataFrame(real_value_needed,columns=feature)


In [10]:
real_value_df

,route,time,label
0,B_3,2016-10-18 08:00:00,126.4
1,B_3,2016-10-18 08:20:00,115.52
2,B_3,2016-10-18 08:40:00,98.86
3,B_3,2016-10-18 09:00:00,131.15
4,B_3,2016-10-18 09:20:00,151.95
...,...,...,...
456,B_1,2016-10-24 17:00:00,114.14
457,B_1,2016-10-24 17:20:00,128.66
458,B_1,2016-10-24 17:40:00,123.62
459,B_1,2016-10-24 18:00:00,146.24


In [11]:
real_value_df_A_2= real_value_df.loc[real_value_df['route'] == 'A_2']
real_value_df_A_2.to_csv('real_value_df_A_2.csv', index=False)

In [12]:
real_value_df_A_3= real_value_df.loc[real_value_df['route'] == 'A_3']
real_value_df_A_3.to_csv('real_value_df_A_3.csv', index=False)

In [13]:
real_value_df_B_1= real_value_df.loc[real_value_df['route'] == 'B_1']
real_value_df_B_1.to_csv('real_value_df_B_1.csv', index=False)

In [14]:
real_value_df_B_3= real_value_df.loc[real_value_df['route'] == 'B_3']
real_value_df_B_3.to_csv('real_value_df_B_3.csv', index=False)

In [15]:
real_value_df_C_1= real_value_df.loc[real_value_df['route'] == 'C_1']
real_value_df_C_1.to_csv('real_value_df_C_1.csv', index=False)

In [16]:
real_value_df_C_3= real_value_df.loc[real_value_df['route'] == 'C_3']
real_value_df_C_3.to_csv('real_value_df_C_3.csv', index=False)

## B1

In [43]:
#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
B_1_travel_times=travel_times['B_1']
B_1_travel_times_need={}
for i in B_1_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in B_1_travel_times_need:
            B_1_travel_times_need[i]=[]
        if i in B_1_travel_times_need:
            B_1_travel_times_need[i].extend(B_1_travel_times[i])
        
B_1_ratio_8_9={}
B_1_ratio_9_10={}
B_1_ratio_17_18={}
B_1_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, \
          251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, \
          266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, \
          281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, \
          299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,310,311,312,313,314,315,316,317,\
          318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    B_1_ratio_8_9[i]={}
    B_1_ratio_9_10[i]={}
    B_1_ratio_17_18[i]={}
    B_1_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        B_1_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        B_1_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        B_1_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        B_1_ratio_18_19[i][j]=[]

for i in B_1_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        B_1_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        B_1_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        B_1_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        B_1_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(B_1_travel_times_need[i])
        
for i in date_key:
    k=0
    if B_1_ratio_8_9[i][480]==[]:k+=1
    if B_1_ratio_8_9[i][500]==[]:k+=1
    if B_1_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del B_1_ratio_8_9[i]
        
for i in date_key:
    k=0
    if B_1_ratio_9_10[i][540]==[]:k+=1
    if B_1_ratio_9_10[i][560]==[]:k+=1
    if B_1_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del B_1_ratio_9_10[i]
        
for i in date_key:
    k=0
    if B_1_ratio_17_18[i][17*60]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del B_1_ratio_17_18[i]
        
for i in date_key:
    k=0
    if B_1_ratio_18_19[i][18*60]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del B_1_ratio_18_19[i]
        
for i in B_1_ratio_8_9:
    for k in [480,500,520]:
        if B_1_ratio_8_9[i][k]==[] and B_1_ratio_8_9[i][k-20]!=[] and B_1_ratio_8_9[i][k+20]!=[]:
            B_1_ratio_8_9[i][k]=[np.mean([np.mean(B_1_ratio_8_9[i][k-20]),np.mean(B_1_ratio_8_9[i][k+20])])]
            
for i in B_1_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if B_1_ratio_9_10[i][k]==[] and B_1_ratio_9_10[i][k-20]!=[] and B_1_ratio_9_10[i][k+20]!=[]:
            B_1_ratio_9_10[i][k]=[np.mean([np.mean(B_1_ratio_9_10[i][k-20]),np.mean(B_1_ratio_9_10[i][k+20])])]
            
for i in B_1_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if B_1_ratio_17_18[i][k]==[] and B_1_ratio_17_18[i][k-20]!=[] and B_1_ratio_17_18[i][k+20]!=[]:
            B_1_ratio_17_18[i][k]=[np.mean([np.mean(B_1_ratio_17_18[i][k-20]),np.mean(B_1_ratio_17_18[i][k+20])])]
            
for i in B_1_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if B_1_ratio_18_19[i][k]==[] and B_1_ratio_18_19[i][k-20]!=[] and B_1_ratio_18_19[i][k+20]!=[]:
            B_1_ratio_18_19[i][k]=[np.mean([np.mean(B_1_ratio_18_19[i][k-20]),np.mean(B_1_ratio_18_19[i][k+20])])]
            
B_1_ratio_8_9_key=list(B_1_ratio_8_9.keys())
for i in B_1_ratio_8_9_key:
    k=0
    if B_1_ratio_8_9[i][480]==[]:k+=1
    if B_1_ratio_8_9[i][500]==[]:k+=1
    if B_1_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del B_1_ratio_8_9[i]
        
B_1_ratio_9_10_key=list(B_1_ratio_9_10.keys())
for i in B_1_ratio_9_10_key:
    k=0
    if B_1_ratio_9_10[i][540]==[]:k+=1
    if B_1_ratio_9_10[i][560]==[]:k+=1
    if B_1_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del B_1_ratio_9_10[i]

B_1_ratio_17_18_key=list(B_1_ratio_17_18.keys())
for i in B_1_ratio_17_18_key:
    k=0
    if B_1_ratio_17_18[i][17*60]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del B_1_ratio_17_18[i]

B_1_ratio_18_19_key=list(B_1_ratio_18_19.keys())
for i in  B_1_ratio_18_19_key:
    k=0
    if B_1_ratio_18_19[i][18*60]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del B_1_ratio_18_19[i]
        
for i in B_1_ratio_8_9:
    del B_1_ratio_8_9[i][8*60-20]
    del B_1_ratio_8_9[i][9*60]
    
for i in B_1_ratio_9_10:
    del B_1_ratio_9_10[i][9*60-20]
    del B_1_ratio_9_10[i][10*60]
    
for i in B_1_ratio_17_18:
    del B_1_ratio_17_18[i][17*60-20]
    del B_1_ratio_17_18[i][18*60]
    
for i in B_1_ratio_18_19:
    del B_1_ratio_18_19[i][18*60-20]
    del B_1_ratio_18_19[i][19*60]
    
for i in B_1_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in B_1_ratio_8_9[i]: 
        if B_1_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_8_9[i][j]))
            temp2.append(np.mean(B_1_ratio_8_9[i][j]))
        if B_1_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in B_1_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in B_1_ratio_9_10[i]: 
        if B_1_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_9_10[i][j]))
            temp2.append(np.mean(B_1_ratio_9_10[i][j]))
        if B_1_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in B_1_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in B_1_ratio_17_18[i]: 
        if B_1_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_17_18[i][j]))
            temp2.append(np.mean(B_1_ratio_17_18[i][j]))
        if B_1_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in B_1_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in B_1_ratio_18_19[i]: 
        if B_1_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(B_1_ratio_18_19[i][j]))
            temp2.append(np.mean(B_1_ratio_18_19[i][j]))
        if B_1_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_1_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
B_1_ratio_8_9_480=[]
B_1_ratio_8_9_500=[]
B_1_ratio_8_9_520=[]

for i in B_1_ratio_8_9:
    for j in B_1_ratio_8_9[i]:
        if j==480: B_1_ratio_8_9_480.extend(B_1_ratio_8_9[i][j])
        if j==500: B_1_ratio_8_9_500.extend(B_1_ratio_8_9[i][j])
        if j==520: B_1_ratio_8_9_520.extend(B_1_ratio_8_9[i][j])
    
    
B_1_ratio_9_10_540=[]
B_1_ratio_9_10_560=[]
B_1_ratio_9_10_580=[]

for i in B_1_ratio_9_10:
    for j in B_1_ratio_9_10[i]:
        if j==540: B_1_ratio_9_10_540.extend(B_1_ratio_9_10[i][j])
        if j==560: B_1_ratio_9_10_560.extend(B_1_ratio_9_10[i][j])
        if j==580: B_1_ratio_9_10_580.extend(B_1_ratio_9_10[i][j])
            
B_1_ratio_17_18_1020=[]
B_1_ratio_17_18_1040=[]
B_1_ratio_17_18_1060=[]

for i in B_1_ratio_17_18:
    for j in B_1_ratio_17_18[i]:
        if j==1020: B_1_ratio_17_18_1020.extend(B_1_ratio_17_18[i][j])
        if j==1040: B_1_ratio_17_18_1040.extend(B_1_ratio_17_18[i][j])
        if j==1060: B_1_ratio_17_18_1060.extend(B_1_ratio_17_18[i][j])
    
B_1_ratio_18_19_1080=[]
B_1_ratio_18_19_1100=[]
B_1_ratio_18_19_1120=[]

for i in B_1_ratio_18_19:
    for j in B_1_ratio_18_19[i]:
        if j==1080: B_1_ratio_18_19_1080.extend(B_1_ratio_18_19[i][j])
        if j==1100: B_1_ratio_18_19_1100.extend(B_1_ratio_18_19[i][j])
        if j==1120: B_1_ratio_18_19_1120.extend(B_1_ratio_18_19[i][j])
            
B_1_ratio_8_9_480.sort()
B_1_ratio_8_9_480_used=B_1_ratio_8_9_480[5:len(B_1_ratio_8_9_480)-5]

B_1_ratio_8_9_500.sort()
B_1_ratio_8_9_500_used=B_1_ratio_8_9_500[5:len(B_1_ratio_8_9_500)-5]

B_1_ratio_8_9_520.sort()
B_1_ratio_8_9_520_used=B_1_ratio_8_9_520[5:len(B_1_ratio_8_9_520)-5]

B_1_ratio_9_10_540.sort()
B_1_ratio_9_10_540_used=B_1_ratio_9_10_540[5:len(B_1_ratio_9_10_540)-5]

B_1_ratio_9_10_560.sort()
B_1_ratio_9_10_560_used=B_1_ratio_9_10_560[5:len(B_1_ratio_9_10_560)-5]

B_1_ratio_9_10_580.sort()
B_1_ratio_9_10_580_used=B_1_ratio_9_10_580[5:len(B_1_ratio_9_10_580)-5]

B_1_ratio_17_18_1020.sort()
B_1_ratio_17_18_1020_used=B_1_ratio_17_18_1020[5:len(B_1_ratio_17_18_1020)-5]

B_1_ratio_17_18_1040.sort()
B_1_ratio_17_18_1040_used=B_1_ratio_17_18_1040[5:len(B_1_ratio_17_18_1040)-5]

B_1_ratio_17_18_1060.sort()
B_1_ratio_17_18_1060_used=B_1_ratio_17_18_1060[5:len(B_1_ratio_17_18_1060)-5]

B_1_ratio_18_19_1080.sort()
B_1_ratio_18_19_1080_used=B_1_ratio_18_19_1080[5:len(B_1_ratio_18_19_1080)-5]

B_1_ratio_18_19_1100.sort()
B_1_ratio_18_19_1100_used=B_1_ratio_18_19_1100[5:len(B_1_ratio_18_19_1100)-5]

B_1_ratio_18_19_1120.sort()
B_1_ratio_18_19_1120_used=B_1_ratio_18_19_1120[5:len(B_1_ratio_18_19_1120)-5]

names=locals()
B_1_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    B_1_ratio.append(np.mean(names['B_1_ratio_'+str(i)+'_used']))
    
B_1_ratio_8=B_1_ratio[0:3]
B_1_ratio_9=B_1_ratio[3:6]
B_1_ratio_17=B_1_ratio[6:9]
B_1_ratio_18=B_1_ratio[9:12]


In [44]:
print(B_1_ratio_8)
print(B_1_ratio_9)
print(B_1_ratio_17)
print(B_1_ratio_18)

[1.0031420374343412, 0.9934482062307293, 0.9873723329424676]
[1.0143332259275053, 0.9560381921561566, 0.9965976774126419]
[0.9289831850456122, 1.000556880529891, 1.0657150770402124]
[1.0174539659659674, 1.0053514813377316, 0.9790868479720023]


In [45]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_train/B_1.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_test/B_1.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_p5= pd.read_csv('/Users/vayne/Desktop/dm_pro_engin/used_data_train/5min_shifted_with_10.1_B_1.csv')
train_m5= pd.read_csv('/Users/vayne/Desktop/dm_pro_engin/used_data_train/-5min_shifted_with_10.1_B_1.csv')

train_p5.reset_index(drop=True, inplace=True)
train_m5.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)
#train1 = pd.concat([train,train_p5,train_m5])
train1 = pd.concat([train,train_p5])
train1.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train1)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=5, learning_rate=0.1, 
                                 max_depth=3, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
#print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

0.22132865969004645
0.13766766452545837
         label           0
0   111.033333  126.211375
1   127.316667  146.082444
2   110.166667  127.038939
3   135.886667  127.038939
4   109.650000  127.276838
5   125.720000  127.038939
6   112.630000  127.038939
7   111.406667  126.211375
8   132.586667  146.082444
9   119.023333  127.038939
10  111.993333  127.038939
11  133.820000  127.276838
12  148.145000  127.038939
13  110.536667  127.038939
14  141.666667  118.883816
15  117.930000  127.862763
16  133.916667  128.075112
17  198.110000  130.077236
18  142.356667  128.075112
19  105.203333  127.862763
20  122.140000  127.862763
21  146.043333  128.993448
22  141.590000  129.096955
23  100.986667  128.075112
24  290.595000  131.311427
25  145.325000  128.075112
26  118.373333  129.096955
27  136.015000  129.096955


In [46]:


test_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_test/B_1.csv'

test = pd.read_csv(test_path)
test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in B_1_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in B_1_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in B_1_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in B_1_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])
    


In [47]:
final_result.sort(key = lambda x: x[0]) 

In [48]:
real_value_df_B_1.loc[real_value_df_B_1['route'] == 'B_1']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_B_1_time=list(real_value_df_B_1['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_B_1_time:final_result3.append(i[0])
    

In [49]:
pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    

In [50]:


#load the real value 
real_value_df_B_1=real_value_df.loc[real_value_df['route'] =='B_1']

real_value=list(real_value_df_B_1['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))

In [51]:
mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.22866655856415827

In [52]:
mape_list

[0.42721154343363893,
 0.07852969610279423,
 0.15045808648751996,
 0.1667917545325613,
 0.003691727103775401,
 0.21657765103168145,
 0.07965778457227254,
 0.21104981175742032,
 0.17852379619359662,
 0.0005719198551892949,
 0.013736753667372294,
 0.2796270326491491,
 0.5874925863929779,
 0.0697725349742941,
 0.06326949165887295,
 0.25786313088175206,
 0.012618879083633548,
 0.02496074727274826,
 0.011794037673625437,
 0.1781538487841099,
 0.07232001835284914,
 0.17001560774420807,
 0.08658459893192474,
 0.5038332750291518,
 0.0805854650451327,
 0.15319101285166437,
 0.04447764948978889,
 0.35805987482049084,
 0.2624241124040403,
 0.027837763261571648,
 5.005093586339683,
 0.10427475654108805,
 0.08809062698946701,
 0.00630211619945628,
 0.22572634376947273,
 0.06282607583614257,
 0.047129991115163916,
 0.05474665630514352,
 0.2949443739654595,
 0.30735088522747545,
 0.4131057313836134,
 0.6597393795464044,
 0.31810071289528535,
 0.21400348309780295,
 0.08637293699049878,
 0.170217228771

In [53]:
mape_list.index(5.005093586339683)

30

In [54]:
mape_list.remove(5.005093586339683)

In [55]:
np.mean(mape_list)

0.16232729428949813

In [221]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare[30]

[21.7, 130.31053082357113]

In [222]:
test_compare

[[88.71, 126.6079360179981],
 [136.07, 125.38446425129278],
 [108.32, 124.61761992832815],
 [109.72, 128.02039130731262],
 [121.11, 120.66289493046176],
 [103.39, 125.78196334016555],
 [120.0, 110.4410658513273],
 [150.77, 118.95001988133374],
 [154.23, 126.69627491306159],
 [131.32, 131.24489548461653],
 [131.49, 129.68375426027723],
 [175.32, 126.29578863595117],
 [92.31, 146.5414406499358],
 [135.66, 145.12534209461273],
 [153.98, 144.23776367436673],
 [117.8, 148.1762768178704],
 [137.92, 139.66039580321473],
 [142.04, 145.58542454262115],
 [120.2, 118.78235667163023],
 [115.66, 136.26527415037015],
 [141.59, 131.3502086014201],
 [108.92, 127.43809999549914],
 [138.17, 126.20660596557595],
 [83.41, 125.43473347018154],
 [119.25, 128.85981670663207],
 [105.32, 121.45407747353728],
 [132.5, 126.60671144260297],
 [87.61, 118.9796256330232],
 [173.74, 128.14643471092205],
 [140.4, 136.49157803807535],
 [21.7, 130.31053082357113],
 [143.75, 128.7605037472186],
 [137.51, 125.396657882678

## A2

In [32]:
#A2

#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    #國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
A_2_travel_times=travel_times['A_2']
A_2_travel_times_need={}
for i in A_2_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in A_2_travel_times_need:
            A_2_travel_times_need[i]=[]
        if i in A_2_travel_times_need:
            A_2_travel_times_need[i].extend(A_2_travel_times[i])
        
A_2_ratio_8_9={}
A_2_ratio_9_10={}
A_2_ratio_17_18={}
A_2_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, \
          251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, \
          266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, \
          281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, \
          299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,310,311,312,313,314,315,316,317,\
          318, 319, 320, 321, 322, 323, 324, 325, 326, 327]


for i in date_key:
    A_2_ratio_8_9[i]={}
    A_2_ratio_9_10[i]={}
    A_2_ratio_17_18[i]={}
    A_2_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        A_2_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        A_2_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        A_2_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        A_2_ratio_18_19[i][j]=[]

for i in A_2_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        A_2_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        A_2_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        A_2_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        A_2_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(A_2_travel_times_need[i])
        
for i in date_key:
    k=0
    if A_2_ratio_8_9[i][480]==[]:k+=1
    if A_2_ratio_8_9[i][500]==[]:k+=1
    if A_2_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del A_2_ratio_8_9[i]
        
for i in date_key:
    k=0
    if A_2_ratio_9_10[i][540]==[]:k+=1
    if A_2_ratio_9_10[i][560]==[]:k+=1
    if A_2_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del A_2_ratio_9_10[i]
        
for i in date_key:
    k=0
    if A_2_ratio_17_18[i][17*60]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del A_2_ratio_17_18[i]
        
for i in date_key:
    k=0
    if A_2_ratio_18_19[i][18*60]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del A_2_ratio_18_19[i]
        
for i in A_2_ratio_8_9:
    for k in [480,500,520]:
        if A_2_ratio_8_9[i][k]==[] and A_2_ratio_8_9[i][k-20]!=[] and A_2_ratio_8_9[i][k+20]!=[]:
            A_2_ratio_8_9[i][k]=[np.mean([np.mean(A_2_ratio_8_9[i][k-20]),np.mean(A_2_ratio_8_9[i][k+20])])]
            
for i in A_2_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if A_2_ratio_9_10[i][k]==[] and A_2_ratio_9_10[i][k-20]!=[] and A_2_ratio_9_10[i][k+20]!=[]:
            A_2_ratio_9_10[i][k]=[np.mean([np.mean(A_2_ratio_9_10[i][k-20]),np.mean(A_2_ratio_9_10[i][k+20])])]
            
for i in A_2_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if A_2_ratio_17_18[i][k]==[] and A_2_ratio_17_18[i][k-20]!=[] and A_2_ratio_17_18[i][k+20]!=[]:
            A_2_ratio_17_18[i][k]=[np.mean([np.mean(A_2_ratio_17_18[i][k-20]),np.mean(A_2_ratio_17_18[i][k+20])])]
            
for i in A_2_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if A_2_ratio_18_19[i][k]==[] and A_2_ratio_18_19[i][k-20]!=[] and A_2_ratio_18_19[i][k+20]!=[]:
            A_2_ratio_18_19[i][k]=[np.mean([np.mean(A_2_ratio_18_19[i][k-20]),np.mean(A_2_ratio_18_19[i][k+20])])]
            
A_2_ratio_8_9_key=list(A_2_ratio_8_9.keys())
for i in A_2_ratio_8_9_key:
    k=0
    if A_2_ratio_8_9[i][480]==[]:k+=1
    if A_2_ratio_8_9[i][500]==[]:k+=1
    if A_2_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del A_2_ratio_8_9[i]
        
A_2_ratio_9_10_key=list(A_2_ratio_9_10.keys())
for i in A_2_ratio_9_10_key:
    k=0
    if A_2_ratio_9_10[i][540]==[]:k+=1
    if A_2_ratio_9_10[i][560]==[]:k+=1
    if A_2_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del A_2_ratio_9_10[i]

A_2_ratio_17_18_key=list(A_2_ratio_17_18.keys())
for i in A_2_ratio_17_18_key:
    k=0
    if A_2_ratio_17_18[i][17*60]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_2_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del A_2_ratio_17_18[i]

A_2_ratio_18_19_key=list(A_2_ratio_18_19.keys())
for i in  A_2_ratio_18_19_key:
    k=0
    if A_2_ratio_18_19[i][18*60]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_2_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del A_2_ratio_18_19[i]
        
for i in A_2_ratio_8_9:
    del A_2_ratio_8_9[i][8*60-20]
    del A_2_ratio_8_9[i][9*60]
    
for i in A_2_ratio_9_10:
    del A_2_ratio_9_10[i][9*60-20]
    del A_2_ratio_9_10[i][10*60]
    
for i in A_2_ratio_17_18:
    del A_2_ratio_17_18[i][17*60-20]
    del A_2_ratio_17_18[i][18*60]
    
for i in A_2_ratio_18_19:
    del A_2_ratio_18_19[i][18*60-20]
    del A_2_ratio_18_19[i][19*60]
    
for i in A_2_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in A_2_ratio_8_9[i]: 
        if A_2_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_8_9[i][j]))
            temp2.append(np.mean(A_2_ratio_8_9[i][j]))
        if A_2_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in A_2_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in A_2_ratio_9_10[i]: 
        if A_2_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_9_10[i][j]))
            temp2.append(np.mean(A_2_ratio_9_10[i][j]))
        if A_2_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in A_2_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in A_2_ratio_17_18[i]: 
        if A_2_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_17_18[i][j]))
            temp2.append(np.mean(A_2_ratio_17_18[i][j]))
        if A_2_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in A_2_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in A_2_ratio_18_19[i]: 
        if A_2_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(A_2_ratio_18_19[i][j]))
            temp2.append(np.mean(A_2_ratio_18_19[i][j]))
        if A_2_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_2_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
A_2_ratio_8_9_480=[]
A_2_ratio_8_9_500=[]
A_2_ratio_8_9_520=[]

for i in A_2_ratio_8_9:
    for j in A_2_ratio_8_9[i]:
        if j==480: A_2_ratio_8_9_480.extend(A_2_ratio_8_9[i][j])
        if j==500: A_2_ratio_8_9_500.extend(A_2_ratio_8_9[i][j])
        if j==520: A_2_ratio_8_9_520.extend(A_2_ratio_8_9[i][j])
    
    
A_2_ratio_9_10_540=[]
A_2_ratio_9_10_560=[]
A_2_ratio_9_10_580=[]

for i in A_2_ratio_9_10:
    for j in A_2_ratio_9_10[i]:
        if j==540: A_2_ratio_9_10_540.extend(A_2_ratio_9_10[i][j])
        if j==560: A_2_ratio_9_10_560.extend(A_2_ratio_9_10[i][j])
        if j==580: A_2_ratio_9_10_580.extend(A_2_ratio_9_10[i][j])
            
A_2_ratio_17_18_1020=[]
A_2_ratio_17_18_1040=[]
A_2_ratio_17_18_1060=[]

for i in A_2_ratio_17_18:
    for j in A_2_ratio_17_18[i]:
        if j==1020: A_2_ratio_17_18_1020.extend(A_2_ratio_17_18[i][j])
        if j==1040: A_2_ratio_17_18_1040.extend(A_2_ratio_17_18[i][j])
        if j==1060: A_2_ratio_17_18_1060.extend(A_2_ratio_17_18[i][j])
    
A_2_ratio_18_19_1080=[]
A_2_ratio_18_19_1100=[]
A_2_ratio_18_19_1120=[]

for i in A_2_ratio_18_19:
    for j in A_2_ratio_18_19[i]:
        if j==1080: A_2_ratio_18_19_1080.extend(A_2_ratio_18_19[i][j])
        if j==1100: A_2_ratio_18_19_1100.extend(A_2_ratio_18_19[i][j])
        if j==1120: A_2_ratio_18_19_1120.extend(A_2_ratio_18_19[i][j])
            
A_2_ratio_8_9_480.sort()
A_2_ratio_8_9_480_used=A_2_ratio_8_9_480[5:len(A_2_ratio_8_9_480)-5]

A_2_ratio_8_9_500.sort()
A_2_ratio_8_9_500_used=A_2_ratio_8_9_500[5:len(A_2_ratio_8_9_500)-5]

A_2_ratio_8_9_520.sort()
A_2_ratio_8_9_520_used=A_2_ratio_8_9_520[5:len(A_2_ratio_8_9_520)-5]

A_2_ratio_9_10_540.sort()
A_2_ratio_9_10_540_used=A_2_ratio_9_10_540[5:len(A_2_ratio_9_10_540)-5]

A_2_ratio_9_10_560.sort()
A_2_ratio_9_10_560_used=A_2_ratio_9_10_560[5:len(A_2_ratio_9_10_560)-5]

A_2_ratio_9_10_580.sort()
A_2_ratio_9_10_580_used=A_2_ratio_9_10_580[5:len(A_2_ratio_9_10_580)-5]

A_2_ratio_17_18_1020.sort()
A_2_ratio_17_18_1020_used=A_2_ratio_17_18_1020[5:len(A_2_ratio_17_18_1020)-5]

A_2_ratio_17_18_1040.sort()
A_2_ratio_17_18_1040_used=A_2_ratio_17_18_1040[5:len(A_2_ratio_17_18_1040)-5]

A_2_ratio_17_18_1060.sort()
A_2_ratio_17_18_1060_used=A_2_ratio_17_18_1060[5:len(A_2_ratio_17_18_1060)-5]

A_2_ratio_18_19_1080.sort()
A_2_ratio_18_19_1080_used=A_2_ratio_18_19_1080[5:len(A_2_ratio_18_19_1080)-5]

A_2_ratio_18_19_1100.sort()
A_2_ratio_18_19_1100_used=A_2_ratio_18_19_1100[5:len(A_2_ratio_18_19_1100)-5]

A_2_ratio_18_19_1120.sort()
A_2_ratio_18_19_1120_used=A_2_ratio_18_19_1120[5:len(A_2_ratio_18_19_1120)-5]

names=locals()
A_2_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    A_2_ratio.append(np.mean(names['A_2_ratio_'+str(i)+'_used']))
    
A_2_ratio_8=A_2_ratio[0:3]
A_2_ratio_9=A_2_ratio[3:6]
A_2_ratio_17=A_2_ratio[6:9]
A_2_ratio_18=A_2_ratio[9:12]






In [33]:
print(A_2_ratio_8)
print(A_2_ratio_9)
print(A_2_ratio_17)
print(A_2_ratio_18)


[0.9594377262857952, 1.0144147364368832, 1.004964586594884]
[1.000155698203997, 1.0194005415982434, 0.9602155822869323]
[1.0152742931390726, 0.9915728865263712, 0.9819021063140361]
[1.0065583872672326, 0.969453070970589, 1.0087499022367605]


In [34]:
#A_2
train_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_train/A_2.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_test/A_2.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=24, learning_rate=0.1, 
                                 max_depth=5, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in A_2_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in A_2_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in A_2_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in A_2_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])

0.09389583779907343
0.08599277770115589
          label           0
0     75.086667   75.390721
1    148.053333  118.627196
2    109.863333  105.689625
3     74.633333   86.947980
4     88.636667   93.426367
..          ...         ...
346   67.673333   69.979219
347   67.823333   70.793016
348   69.840000   70.705068
349   62.610000   68.002858
350   58.156667   63.816261

[351 rows x 2 columns]
        label          0
0   75.250000  75.934056
1   95.980000  92.867044
2   80.773333  89.206273
3   62.000000  76.330491
4   63.230000  69.450465
5   62.546667  64.078618
6   73.080000  86.389733
7   80.073333  75.642101
8   80.103333  77.778056
9   89.450000  76.016082
10  85.500000  74.557226
11  71.763333  71.737976
12  71.486667  64.544145
13  68.963333  75.196225
14  68.896667  79.095305
15  61.296667  65.473227
16  77.910000  73.558679
17  81.890000  78.574561
18  70.700000  74.627194
19  75.013333  72.470010
20  69.283333  72.493795
21  66.796667  80.816468
22  67.853333  74.212730


In [35]:
final_result.sort(key = lambda x: x[0]) 

In [37]:
final_result1=final_result

In [38]:
pred_value=[]
for i in final_result1:
    pred_value.append(i[2])




In [39]:
#load the real value 
real_value_df_A_2=real_value_df.loc[real_value_df['route'] =='A_2']

real_value=list(real_value_df_A_2['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))

In [41]:
mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.13194938082850158

In [42]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

[[77.77, 72.85399764771233],
 [70.3, 77.02862499297115],
 [77.68, 76.31103678948783],
 [79.28, 75.65387877600898],
 [83.27, 77.10959917216938],
 [77.67, 72.63272447641806],
 [75.0, 80.30342970556717],
 [63.3, 78.4287597245501],
 [68.39, 77.66384641567657],
 [88.83, 81.34649324447727],
 [59.33, 78.34777265395702],
 [52.23, 81.52360374290338],
 [91.72, 89.10014531947603],
 [92.28, 94.20569772740075],
 [103.94, 93.32809024841139],
 [79.27, 77.79016590565102],
 [81.22, 79.28699241291885],
 [79.82, 74.68370133312635],
 [60.0, 66.47328466825546],
 [56.98, 64.92147708339783],
 [66.91, 64.28829989141842],
 [63.8, 74.69944576099215],
 [55.74, 71.94575894340895],
 [84.02, 74.86208406958059],
 [87.24, 85.58786383008845],
 [75.48, 90.49215801165276],
 [79.6, 89.649145364045],
 [103.35, 76.02791775951687],
 [97.05, 77.49083535674693],
 [67.95, 72.99182662520548],
 [96.52, 74.68223627384887],
 [54.93, 72.93879210252102],
 [82.28, 72.22742228093612],
 [75.25, 67.54787054388096],
 [50.34, 65.057816183

## A3

In [23]:
#A3

#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
A_3_travel_times=travel_times['A_3']
A_3_travel_times_need={}
for i in A_3_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in A_3_travel_times_need:
            A_3_travel_times_need[i]=[]
        if i in A_3_travel_times_need:
            A_3_travel_times_need[i].extend(A_3_travel_times[i])
        
A_3_ratio_8_9={}
A_3_ratio_9_10={}
A_3_ratio_17_18={}
A_3_ratio_18_19={}

#date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]
date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, \
          251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, \
          266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, \
          281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, \
          299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,310,311,312,313,314,315,316,317,\
          318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    A_3_ratio_8_9[i]={}
    A_3_ratio_9_10[i]={}
    A_3_ratio_17_18[i]={}
    A_3_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        A_3_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        A_3_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        A_3_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        A_3_ratio_18_19[i][j]=[]

for i in A_3_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        A_3_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        A_3_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        A_3_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        A_3_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(A_3_travel_times_need[i])
        
for i in date_key:
    k=0
    if A_3_ratio_8_9[i][480]==[]:k+=1
    if A_3_ratio_8_9[i][500]==[]:k+=1
    if A_3_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del A_3_ratio_8_9[i]
        
for i in date_key:
    k=0
    if A_3_ratio_9_10[i][540]==[]:k+=1
    if A_3_ratio_9_10[i][560]==[]:k+=1
    if A_3_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del A_3_ratio_9_10[i]
        
for i in date_key:
    k=0
    if A_3_ratio_17_18[i][17*60]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del A_3_ratio_17_18[i]
        
for i in date_key:
    k=0
    if A_3_ratio_18_19[i][18*60]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del A_3_ratio_18_19[i]
        
for i in A_3_ratio_8_9:
    for k in [480,500,520]:
        if A_3_ratio_8_9[i][k]==[] and A_3_ratio_8_9[i][k-20]!=[] and A_3_ratio_8_9[i][k+20]!=[]:
            A_3_ratio_8_9[i][k]=[np.mean([np.mean(A_3_ratio_8_9[i][k-20]),np.mean(A_3_ratio_8_9[i][k+20])])]
            
for i in A_3_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if A_3_ratio_9_10[i][k]==[] and A_3_ratio_9_10[i][k-20]!=[] and A_3_ratio_9_10[i][k+20]!=[]:
            A_3_ratio_9_10[i][k]=[np.mean([np.mean(A_3_ratio_9_10[i][k-20]),np.mean(A_3_ratio_9_10[i][k+20])])]
            
for i in A_3_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if A_3_ratio_17_18[i][k]==[] and A_3_ratio_17_18[i][k-20]!=[] and A_3_ratio_17_18[i][k+20]!=[]:
            A_3_ratio_17_18[i][k]=[np.mean([np.mean(A_3_ratio_17_18[i][k-20]),np.mean(A_3_ratio_17_18[i][k+20])])]
            
for i in A_3_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if A_3_ratio_18_19[i][k]==[] and A_3_ratio_18_19[i][k-20]!=[] and A_3_ratio_18_19[i][k+20]!=[]:
            A_3_ratio_18_19[i][k]=[np.mean([np.mean(A_3_ratio_18_19[i][k-20]),np.mean(A_3_ratio_18_19[i][k+20])])]
            
A_3_ratio_8_9_key=list(A_3_ratio_8_9.keys())
for i in A_3_ratio_8_9_key:
    k=0
    if A_3_ratio_8_9[i][480]==[]:k+=1
    if A_3_ratio_8_9[i][500]==[]:k+=1
    if A_3_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del A_3_ratio_8_9[i]
        
A_3_ratio_9_10_key=list(A_3_ratio_9_10.keys())
for i in A_3_ratio_9_10_key:
    k=0
    if A_3_ratio_9_10[i][540]==[]:k+=1
    if A_3_ratio_9_10[i][560]==[]:k+=1
    if A_3_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del A_3_ratio_9_10[i]

A_3_ratio_17_18_key=list(A_3_ratio_17_18.keys())
for i in A_3_ratio_17_18_key:
    k=0
    if A_3_ratio_17_18[i][17*60]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if A_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del A_3_ratio_17_18[i]

A_3_ratio_18_19_key=list(A_3_ratio_18_19.keys())
for i in  A_3_ratio_18_19_key:
    k=0
    if A_3_ratio_18_19[i][18*60]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if A_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del A_3_ratio_18_19[i]
        
for i in A_3_ratio_8_9:
    del A_3_ratio_8_9[i][8*60-20]
    del A_3_ratio_8_9[i][9*60]
    
for i in A_3_ratio_9_10:
    del A_3_ratio_9_10[i][9*60-20]
    del A_3_ratio_9_10[i][10*60]
    
for i in A_3_ratio_17_18:
    del A_3_ratio_17_18[i][17*60-20]
    del A_3_ratio_17_18[i][18*60]
    
for i in A_3_ratio_18_19:
    del A_3_ratio_18_19[i][18*60-20]
    del A_3_ratio_18_19[i][19*60]
    
for i in A_3_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in A_3_ratio_8_9[i]: 
        if A_3_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_8_9[i][j]))
            temp2.append(np.mean(A_3_ratio_8_9[i][j]))
        if A_3_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in A_3_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in A_3_ratio_9_10[i]: 
        if A_3_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_9_10[i][j]))
            temp2.append(np.mean(A_3_ratio_9_10[i][j]))
        if A_3_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in A_3_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in A_3_ratio_17_18[i]: 
        if A_3_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_17_18[i][j]))
            temp2.append(np.mean(A_3_ratio_17_18[i][j]))
        if A_3_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in A_3_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in A_3_ratio_18_19[i]: 
        if A_3_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(A_3_ratio_18_19[i][j]))
            temp2.append(np.mean(A_3_ratio_18_19[i][j]))
        if A_3_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        A_3_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
A_3_ratio_8_9_480=[]
A_3_ratio_8_9_500=[]
A_3_ratio_8_9_520=[]

for i in A_3_ratio_8_9:
    for j in A_3_ratio_8_9[i]:
        if j==480: A_3_ratio_8_9_480.extend(A_3_ratio_8_9[i][j])
        if j==500: A_3_ratio_8_9_500.extend(A_3_ratio_8_9[i][j])
        if j==520: A_3_ratio_8_9_520.extend(A_3_ratio_8_9[i][j])
    
    
A_3_ratio_9_10_540=[]
A_3_ratio_9_10_560=[]
A_3_ratio_9_10_580=[]

for i in A_3_ratio_9_10:
    for j in A_3_ratio_9_10[i]:
        if j==540: A_3_ratio_9_10_540.extend(A_3_ratio_9_10[i][j])
        if j==560: A_3_ratio_9_10_560.extend(A_3_ratio_9_10[i][j])
        if j==580: A_3_ratio_9_10_580.extend(A_3_ratio_9_10[i][j])
            
A_3_ratio_17_18_1020=[]
A_3_ratio_17_18_1040=[]
A_3_ratio_17_18_1060=[]

for i in A_3_ratio_17_18:
    for j in A_3_ratio_17_18[i]:
        if j==1020: A_3_ratio_17_18_1020.extend(A_3_ratio_17_18[i][j])
        if j==1040: A_3_ratio_17_18_1040.extend(A_3_ratio_17_18[i][j])
        if j==1060: A_3_ratio_17_18_1060.extend(A_3_ratio_17_18[i][j])
    
A_3_ratio_18_19_1080=[]
A_3_ratio_18_19_1100=[]
A_3_ratio_18_19_1120=[]

for i in A_3_ratio_18_19:
    for j in A_3_ratio_18_19[i]:
        if j==1080: A_3_ratio_18_19_1080.extend(A_3_ratio_18_19[i][j])
        if j==1100: A_3_ratio_18_19_1100.extend(A_3_ratio_18_19[i][j])
        if j==1120: A_3_ratio_18_19_1120.extend(A_3_ratio_18_19[i][j])
            
A_3_ratio_8_9_480.sort()
A_3_ratio_8_9_480_used=A_3_ratio_8_9_480[5:len(A_3_ratio_8_9_480)-5]

A_3_ratio_8_9_500.sort()
A_3_ratio_8_9_500_used=A_3_ratio_8_9_500[5:len(A_3_ratio_8_9_500)-5]

A_3_ratio_8_9_520.sort()
A_3_ratio_8_9_520_used=A_3_ratio_8_9_520[5:len(A_3_ratio_8_9_520)-5]

A_3_ratio_9_10_540.sort()
A_3_ratio_9_10_540_used=A_3_ratio_9_10_540[5:len(A_3_ratio_9_10_540)-5]

A_3_ratio_9_10_560.sort()
A_3_ratio_9_10_560_used=A_3_ratio_9_10_560[5:len(A_3_ratio_9_10_560)-5]

A_3_ratio_9_10_580.sort()
A_3_ratio_9_10_580_used=A_3_ratio_9_10_580[5:len(A_3_ratio_9_10_580)-5]

A_3_ratio_17_18_1020.sort()
A_3_ratio_17_18_1020_used=A_3_ratio_17_18_1020[5:len(A_3_ratio_17_18_1020)-5]

A_3_ratio_17_18_1040.sort()
A_3_ratio_17_18_1040_used=A_3_ratio_17_18_1040[5:len(A_3_ratio_17_18_1040)-5]

A_3_ratio_17_18_1060.sort()
A_3_ratio_17_18_1060_used=A_3_ratio_17_18_1060[5:len(A_3_ratio_17_18_1060)-5]

A_3_ratio_18_19_1080.sort()
A_3_ratio_18_19_1080_used=A_3_ratio_18_19_1080[5:len(A_3_ratio_18_19_1080)-5]

A_3_ratio_18_19_1100.sort()
A_3_ratio_18_19_1100_used=A_3_ratio_18_19_1100[5:len(A_3_ratio_18_19_1100)-5]

A_3_ratio_18_19_1120.sort()
A_3_ratio_18_19_1120_used=A_3_ratio_18_19_1120[5:len(A_3_ratio_18_19_1120)-5]

names=locals()
A_3_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    A_3_ratio.append(np.mean(names['A_3_ratio_'+str(i)+'_used']))
    
A_3_ratio_8=A_3_ratio[0:3]
A_3_ratio_9=A_3_ratio[3:6]
A_3_ratio_17=A_3_ratio[6:9]
A_3_ratio_18=A_3_ratio[9:12]





In [24]:

print(A_3_ratio_8)
print(A_3_ratio_9)
print(A_3_ratio_17)
print(A_3_ratio_18)

[0.9702602614629005, 0.978968720287374, 1.0444125878050396]
[1.057196991804728, 1.0448782719176082, 0.8779080952581202]
[1.0131716009078262, 1.008282644002323, 0.977339452004021]
[0.9618902905147334, 1.0074499810834463, 1.0076270414107436]


In [25]:
print(A_3_ratio_8)
print(A_3_ratio_9)
print(A_3_ratio_17)
print(A_3_ratio_18)

[0.9702602614629005, 0.978968720287374, 1.0444125878050396]
[1.057196991804728, 1.0448782719176082, 0.8779080952581202]
[1.0131716009078262, 1.008282644002323, 0.977339452004021]
[0.9618902905147334, 1.0074499810834463, 1.0076270414107436]


In [26]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_train/A_3.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_test/A_3.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=15, learning_rate=0.1, 
                                 max_depth=4, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in A_3_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in A_3_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in A_3_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in A_3_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])
    

final_result.sort(key = lambda x: x[0]) 

0.14164875849348846
0.16207408138415233
          label           0
0    146.513333  136.240796
1    203.340000  171.820264
2    168.586667  182.326027
3    139.636667  142.596285
4    108.560000  141.327662
..          ...         ...
342  110.613333  122.626933
343  140.670000  125.840289
344  257.630000  168.623405
345  137.190000  137.030973
346  129.810000  129.260001

[347 rows x 2 columns]
         label           0
0   206.073333  198.222112
1   324.796667  220.388315
2   217.020000  142.596285
3   256.776667  142.596285
4   143.293333  142.596285
5    97.700000  121.324113
6   227.373333  219.172405
7   185.443333  197.987002
8   285.266667  218.861371
9   215.743333  146.743359
10  227.686667  145.740784
11  129.380000  145.740784
12  105.200000  124.905823
13  167.313333  232.932007
14  126.723333  131.747854
15  129.363333  121.665255
16  130.300000  139.012502
17  121.636667  125.584284
18  116.326667  125.584284
19  110.940000  126.944840
20  113.813333  133.447189
21  11

In [27]:
final_result1=final_result
pred_value=[]
for i in final_result1:
    pred_value.append(i[2])

In [28]:
#load the real value 
real_value_df_A_3=real_value_df.loc[real_value_df['route'] =='A_3']

real_value=list(real_value_df_A_3['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))


mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.21883943302247055

In [29]:
mape_list

[0.026192211110982862,
 0.16269171603852295,
 0.18436029844135862,
 0.1284507708789643,
 0.15448583318007827,
 0.26890342938824685,
 0.02095361649932556,
 0.07558055430077827,
 0.28595201450335234,
 0.04917347467205128,
 0.1978871948571299,
 0.12029788075411976,
 0.3310788506963255,
 0.3422557576932799,
 0.2954504743556877,
 0.3176052643461809,
 0.27823667751869874,
 0.038770477166609696,
 0.19046574622227694,
 0.004116215003399377,
 0.0556485555921799,
 0.08227205339726208,
 0.24369279015700035,
 0.17069822803800044,
 0.24708582294618275,
 0.42507597275436004,
 0.336588019657613,
 0.23058752378819744,
 0.4175096028370278,
 0.2935944360975858,
 0.008474287359071898,
 0.1095938318718287,
 0.08759527945381018,
 0.023126572657750637,
 0.10862376946354288,
 0.4878860540627528,
 0.4655612284633442,
 0.4841955991150095,
 0.3815482933970248,
 0.35253721602163285,
 0.393035279182513,
 0.3411585256876834,
 0.06394151049525133,
 0.038501224669246804,
 0.14655278561085008,
 0.08478294892816894,
 

In [30]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare


[[197.5, 192.32703830558088],
 [166.9, 194.05324740682948],
 [253.82, 207.02566904961435],
 [240.16, 209.31126286570793],
 [179.19, 206.87231644753822],
 [136.98, 173.81439175760204],
 [136.34, 133.48318392648196],
 [143.7, 132.83907434697815],
 [100.13, 128.76237521222066],
 [122.91, 128.95391177194182],
 [112.75, 135.0617812201414],
 [120.58, 135.08551846133176],
 [319.67, 213.83402379790564],
 [328.02, 215.7532663614503],
 [326.7, 230.1763300279968],
 [339.07, 231.37958301814044],
 [316.84, 228.68349109497547],
 [199.89, 192.14016931916638],
 [152.27, 123.2677808227339],
 [123.18, 122.67296463588127],
 [112.64, 118.90825330190314],
 [110.19, 119.2555575638443],
 [100.43, 124.90406691546755],
 [150.64, 124.9260189283556],
 [183.76, 138.35550917540945],
 [242.81, 139.59730305551383],
 [224.49, 148.92935546706246],
 [201.63, 155.13663757858575],
 [263.23, 153.32894724520918],
 [182.37, 128.82718268888328],
 [139.66, 140.84351897256798],
 [126.32, 140.1638928420494],
 [124.92, 135.86240

In [31]:
real_value_df_A_3

,route,time,label
244,A_3,2016-10-18 08:00:00,197.5
245,A_3,2016-10-18 08:20:00,166.9
246,A_3,2016-10-18 08:40:00,253.82
247,A_3,2016-10-18 09:00:00,240.16
248,A_3,2016-10-18 09:20:00,179.19
...,...,...,...
323,A_3,2016-10-24 17:20:00,124.4
324,A_3,2016-10-24 17:40:00,95.07
325,A_3,2016-10-24 18:00:00,109.61
326,A_3,2016-10-24 18:20:00,109.73


In [129]:
#B3

#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
B_3_travel_times=travel_times['B_3']
B_3_travel_times_need={}
for i in B_3_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in B_3_travel_times_need:
            B_3_travel_times_need[i]=[]
        if i in B_3_travel_times_need:
            B_3_travel_times_need[i].extend(B_3_travel_times[i])
        
B_3_ratio_8_9={}
B_3_ratio_9_10={}
B_3_ratio_17_18={}
B_3_ratio_18_19={}

#date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, \
          251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, \
          266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, \
          281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, \
          299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,310,311,312,313,314,315,316,317,\
          318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    B_3_ratio_8_9[i]={}
    B_3_ratio_9_10[i]={}
    B_3_ratio_17_18[i]={}
    B_3_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        B_3_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        B_3_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        B_3_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        B_3_ratio_18_19[i][j]=[]

for i in B_3_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        B_3_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        B_3_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        B_3_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        B_3_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(B_3_travel_times_need[i])
        
for i in date_key:
    k=0
    if B_3_ratio_8_9[i][480]==[]:k+=1
    if B_3_ratio_8_9[i][500]==[]:k+=1
    if B_3_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del B_3_ratio_8_9[i]
        
for i in date_key:
    k=0
    if B_3_ratio_9_10[i][540]==[]:k+=1
    if B_3_ratio_9_10[i][560]==[]:k+=1
    if B_3_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del B_3_ratio_9_10[i]
        
for i in date_key:
    k=0
    if B_3_ratio_17_18[i][17*60]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del B_3_ratio_17_18[i]
        
for i in date_key:
    k=0
    if B_3_ratio_18_19[i][18*60]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del B_3_ratio_18_19[i]
        
for i in B_3_ratio_8_9:
    for k in [480,500,520]:
        if B_3_ratio_8_9[i][k]==[] and B_3_ratio_8_9[i][k-20]!=[] and B_3_ratio_8_9[i][k+20]!=[]:
            B_3_ratio_8_9[i][k]=[np.mean([np.mean(B_3_ratio_8_9[i][k-20]),np.mean(B_3_ratio_8_9[i][k+20])])]
            
for i in B_3_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if B_3_ratio_9_10[i][k]==[] and B_3_ratio_9_10[i][k-20]!=[] and B_3_ratio_9_10[i][k+20]!=[]:
            B_3_ratio_9_10[i][k]=[np.mean([np.mean(B_3_ratio_9_10[i][k-20]),np.mean(B_3_ratio_9_10[i][k+20])])]
            
for i in B_3_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if B_3_ratio_17_18[i][k]==[] and B_3_ratio_17_18[i][k-20]!=[] and B_3_ratio_17_18[i][k+20]!=[]:
            B_3_ratio_17_18[i][k]=[np.mean([np.mean(B_3_ratio_17_18[i][k-20]),np.mean(B_3_ratio_17_18[i][k+20])])]
            
for i in B_3_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if B_3_ratio_18_19[i][k]==[] and B_3_ratio_18_19[i][k-20]!=[] and B_3_ratio_18_19[i][k+20]!=[]:
            B_3_ratio_18_19[i][k]=[np.mean([np.mean(B_3_ratio_18_19[i][k-20]),np.mean(B_3_ratio_18_19[i][k+20])])]
            
B_3_ratio_8_9_key=list(B_3_ratio_8_9.keys())
for i in B_3_ratio_8_9_key:
    k=0
    if B_3_ratio_8_9[i][480]==[]:k+=1
    if B_3_ratio_8_9[i][500]==[]:k+=1
    if B_3_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del B_3_ratio_8_9[i]
        
B_3_ratio_9_10_key=list(B_3_ratio_9_10.keys())
for i in B_3_ratio_9_10_key:
    k=0
    if B_3_ratio_9_10[i][540]==[]:k+=1
    if B_3_ratio_9_10[i][560]==[]:k+=1
    if B_3_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del B_3_ratio_9_10[i]

B_3_ratio_17_18_key=list(B_3_ratio_17_18.keys())
for i in B_3_ratio_17_18_key:
    k=0
    if B_3_ratio_17_18[i][17*60]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if B_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del B_3_ratio_17_18[i]

B_3_ratio_18_19_key=list(B_3_ratio_18_19.keys())
for i in  B_3_ratio_18_19_key:
    k=0
    if B_3_ratio_18_19[i][18*60]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if B_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del B_3_ratio_18_19[i]
        
for i in B_3_ratio_8_9:
    del B_3_ratio_8_9[i][8*60-20]
    del B_3_ratio_8_9[i][9*60]
    
for i in B_3_ratio_9_10:
    del B_3_ratio_9_10[i][9*60-20]
    del B_3_ratio_9_10[i][10*60]
    
for i in B_3_ratio_17_18:
    del B_3_ratio_17_18[i][17*60-20]
    del B_3_ratio_17_18[i][18*60]
    
for i in B_3_ratio_18_19:
    del B_3_ratio_18_19[i][18*60-20]
    del B_3_ratio_18_19[i][19*60]
    
for i in B_3_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in B_3_ratio_8_9[i]: 
        if B_3_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_8_9[i][j]))
            temp2.append(np.mean(B_3_ratio_8_9[i][j]))
        if B_3_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in B_3_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in B_3_ratio_9_10[i]: 
        if B_3_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_9_10[i][j]))
            temp2.append(np.mean(B_3_ratio_9_10[i][j]))
        if B_3_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in B_3_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in B_3_ratio_17_18[i]: 
        if B_3_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_17_18[i][j]))
            temp2.append(np.mean(B_3_ratio_17_18[i][j]))
        if B_3_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in B_3_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in B_3_ratio_18_19[i]: 
        if B_3_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(B_3_ratio_18_19[i][j]))
            temp2.append(np.mean(B_3_ratio_18_19[i][j]))
        if B_3_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        B_3_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
B_3_ratio_8_9_480=[]
B_3_ratio_8_9_500=[]
B_3_ratio_8_9_520=[]

for i in B_3_ratio_8_9:
    for j in B_3_ratio_8_9[i]:
        if j==480: B_3_ratio_8_9_480.extend(B_3_ratio_8_9[i][j])
        if j==500: B_3_ratio_8_9_500.extend(B_3_ratio_8_9[i][j])
        if j==520: B_3_ratio_8_9_520.extend(B_3_ratio_8_9[i][j])
    
    
B_3_ratio_9_10_540=[]
B_3_ratio_9_10_560=[]
B_3_ratio_9_10_580=[]

for i in B_3_ratio_9_10:
    for j in B_3_ratio_9_10[i]:
        if j==540: B_3_ratio_9_10_540.extend(B_3_ratio_9_10[i][j])
        if j==560: B_3_ratio_9_10_560.extend(B_3_ratio_9_10[i][j])
        if j==580: B_3_ratio_9_10_580.extend(B_3_ratio_9_10[i][j])
            
B_3_ratio_17_18_1020=[]
B_3_ratio_17_18_1040=[]
B_3_ratio_17_18_1060=[]

for i in B_3_ratio_17_18:
    for j in B_3_ratio_17_18[i]:
        if j==1020: B_3_ratio_17_18_1020.extend(B_3_ratio_17_18[i][j])
        if j==1040: B_3_ratio_17_18_1040.extend(B_3_ratio_17_18[i][j])
        if j==1060: B_3_ratio_17_18_1060.extend(B_3_ratio_17_18[i][j])
    
B_3_ratio_18_19_1080=[]
B_3_ratio_18_19_1100=[]
B_3_ratio_18_19_1120=[]

for i in B_3_ratio_18_19:
    for j in B_3_ratio_18_19[i]:
        if j==1080: B_3_ratio_18_19_1080.extend(B_3_ratio_18_19[i][j])
        if j==1100: B_3_ratio_18_19_1100.extend(B_3_ratio_18_19[i][j])
        if j==1120: B_3_ratio_18_19_1120.extend(B_3_ratio_18_19[i][j])
            
B_3_ratio_8_9_480.sort()
B_3_ratio_8_9_480_used=B_3_ratio_8_9_480[5:len(B_3_ratio_8_9_480)-5]

B_3_ratio_8_9_500.sort()
B_3_ratio_8_9_500_used=B_3_ratio_8_9_500[5:len(B_3_ratio_8_9_500)-5]

B_3_ratio_8_9_520.sort()
B_3_ratio_8_9_520_used=B_3_ratio_8_9_520[5:len(B_3_ratio_8_9_520)-5]

B_3_ratio_9_10_540.sort()
B_3_ratio_9_10_540_used=B_3_ratio_9_10_540[5:len(B_3_ratio_9_10_540)-5]

B_3_ratio_9_10_560.sort()
B_3_ratio_9_10_560_used=B_3_ratio_9_10_560[5:len(B_3_ratio_9_10_560)-5]

B_3_ratio_9_10_580.sort()
B_3_ratio_9_10_580_used=B_3_ratio_9_10_580[5:len(B_3_ratio_9_10_580)-5]

B_3_ratio_17_18_1020.sort()
B_3_ratio_17_18_1020_used=B_3_ratio_17_18_1020[5:len(B_3_ratio_17_18_1020)-5]

B_3_ratio_17_18_1040.sort()
B_3_ratio_17_18_1040_used=B_3_ratio_17_18_1040[5:len(B_3_ratio_17_18_1040)-5]

B_3_ratio_17_18_1060.sort()
B_3_ratio_17_18_1060_used=B_3_ratio_17_18_1060[5:len(B_3_ratio_17_18_1060)-5]

B_3_ratio_18_19_1080.sort()
B_3_ratio_18_19_1080_used=B_3_ratio_18_19_1080[5:len(B_3_ratio_18_19_1080)-5]

B_3_ratio_18_19_1100.sort()
B_3_ratio_18_19_1100_used=B_3_ratio_18_19_1100[5:len(B_3_ratio_18_19_1100)-5]

B_3_ratio_18_19_1120.sort()
B_3_ratio_18_19_1120_used=B_3_ratio_18_19_1120[5:len(B_3_ratio_18_19_1120)-5]

names=locals()
B_3_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    B_3_ratio.append(np.mean(names['B_3_ratio_'+str(i)+'_used']))
    
B_3_ratio_8=B_3_ratio[0:3]
B_3_ratio_9=B_3_ratio[3:6]
B_3_ratio_17=B_3_ratio[6:9]
B_3_ratio_18=B_3_ratio[9:12]




In [130]:
print(B_3_ratio_8)
print(B_3_ratio_9)
print(B_3_ratio_17)
print(B_3_ratio_18)

[0.9588587319159432, 0.9877176009992981, 1.0498548948656439]
[1.0254018000740626, 1.0251925941467648, 0.9276552578492043]
[0.9887454541008326, 0.9964707338380407, 1.0048398973940167]
[1.0134617557170276, 0.9490581825784564, 1.0192924016085947]


In [131]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_train/B_3.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_test/B_3.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_x, train_y = get_x_y(train)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=20, learning_rate=0.1, 
                                 max_depth=5, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in B_3_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in B_3_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in B_3_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in B_3_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])
    

final_result.sort(key = lambda x: x[0])

0.10756328601224646
0.13220243072473067
          label           0
0    147.723333  127.267599
1    148.253333  141.846739
2    107.110000  114.405636
3    155.416667  140.574057
4    103.586667  126.071271
..          ...         ...
344  102.470000  108.592797
345  118.543333  112.058245
346   94.883333  103.088640
347  113.926667  106.591125
348   79.440000  101.265893

[349 rows x 2 columns]
         label           0
0   113.593333   97.332757
1   111.976667  103.867678
2    94.890000  104.251552
3    95.633333  105.432438
4    93.423333  115.068698
5    94.453333  101.977422
6   108.876667  107.123063
7   133.596667  101.824463
8   116.496667  106.807329
9    93.263333  113.178156
10   97.956667  109.882123
11  107.416667  127.380681
12   96.690000  105.433921
13  136.133333  112.174408
14   94.173333  101.709626
15  104.376667  100.497580
16   99.193333  141.071738
17  107.746667  104.077517
18  104.146667  101.445199
19   94.710000  111.647408
20   95.560000  108.359930
21  11

In [132]:
final_result1=final_result

In [133]:
pred_value=[]
for i in final_result1:
    pred_value.append(i[2])
    

#load the real value 
real_value_df_B_3=real_value_df.loc[real_value_df['route'] =='B_3']

real_value=list(real_value_df_B_3['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))


mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.20166597372464884

In [134]:
for i in mape_list:
    if i>1:
        print(mape_list.index(i))
        mape_list.remove(i)

np.mean(mape_list)

83


0.18741485382049425

In [135]:
real_value_df_B_3

,route,time,label
0,B_3,2016-10-18 08:00:00,126.4
1,B_3,2016-10-18 08:20:00,115.52
2,B_3,2016-10-18 08:40:00,98.86
3,B_3,2016-10-18 09:00:00,131.15
4,B_3,2016-10-18 09:20:00,151.95
...,...,...,...
79,B_3,2016-10-24 17:20:00,57.83
80,B_3,2016-10-24 17:40:00,118.39
81,B_3,2016-10-24 18:00:00,104.99
82,B_3,2016-10-24 18:20:00,129.71


In [136]:

test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare


[[126.4, 93.32836380166677],
 [115.52, 96.1372770889602],
 [98.86, 102.18527120381984],
 [131.15, 104.41098753875687],
 [151.95, 104.38968525757642],
 [117.69, 94.4579983773773],
 [80.49, 100.56493080952512],
 [97.56, 101.35066612596516],
 [104.47, 102.20189062510151],
 [125.47, 110.05464713786925],
 [111.18, 103.06088296650327],
 [117.39, 110.68781328603662],
 [70.3, 99.59443002256876],
 [145.63, 102.59193374421669],
 [120.0, 109.0459901758643],
 [129.17, 109.52042752402416],
 [108.18, 109.49808279769682],
 [112.14, 99.08038041986622],
 [106.34, 99.36652491228904],
 [105.73, 100.1428968271008],
 [101.06, 100.98397750719978],
 [87.42, 108.82628392301227],
 [110.38, 101.91057990507832],
 [80.84, 109.45238305469711],
 [93.88, 99.9625114214587],
 [87.78, 102.97109332652303],
 [103.01, 109.4490026796577],
 [94.06, 116.05308461200153],
 [114.88, 116.02940707098271],
 [70.85, 104.99031123425563],
 [116.79, 139.4840391741274],
 [93.09, 140.57385780947655],
 [87.7, 141.7545102538997],
 [99.22,

In [137]:
test_compare[83]

[46.32, 110.45045344164232]

In [239]:
#C1

#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
C_1_travel_times=travel_times['C_1']
C_1_travel_times_need={}
for i in C_1_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in C_1_travel_times_need:
            C_1_travel_times_need[i]=[]
        if i in C_1_travel_times_need:
            C_1_travel_times_need[i].extend(C_1_travel_times[i])
        
C_1_ratio_8_9={}
C_1_ratio_9_10={}
C_1_ratio_17_18={}
C_1_ratio_18_19={}


date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    C_1_ratio_8_9[i]={}
    C_1_ratio_9_10[i]={}
    C_1_ratio_17_18[i]={}
    C_1_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        C_1_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        C_1_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        C_1_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        C_1_ratio_18_19[i][j]=[]

for i in C_1_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        C_1_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        C_1_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        C_1_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        C_1_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(C_1_travel_times_need[i])
        
for i in date_key:
    k=0
    if C_1_ratio_8_9[i][480]==[]:k+=1
    if C_1_ratio_8_9[i][500]==[]:k+=1
    if C_1_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del C_1_ratio_8_9[i]
        
for i in date_key:
    k=0
    if C_1_ratio_9_10[i][540]==[]:k+=1
    if C_1_ratio_9_10[i][560]==[]:k+=1
    if C_1_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del C_1_ratio_9_10[i]
        
for i in date_key:
    k=0
    if C_1_ratio_17_18[i][17*60]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del C_1_ratio_17_18[i]
        
for i in date_key:
    k=0
    if C_1_ratio_18_19[i][18*60]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del C_1_ratio_18_19[i]
        
for i in C_1_ratio_8_9:
    for k in [480,500,520]:
        if C_1_ratio_8_9[i][k]==[] and C_1_ratio_8_9[i][k-20]!=[] and C_1_ratio_8_9[i][k+20]!=[]:
            C_1_ratio_8_9[i][k]=[np.mean([np.mean(C_1_ratio_8_9[i][k-20]),np.mean(C_1_ratio_8_9[i][k+20])])]
            
for i in C_1_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if C_1_ratio_9_10[i][k]==[] and C_1_ratio_9_10[i][k-20]!=[] and C_1_ratio_9_10[i][k+20]!=[]:
            C_1_ratio_9_10[i][k]=[np.mean([np.mean(C_1_ratio_9_10[i][k-20]),np.mean(C_1_ratio_9_10[i][k+20])])]
            
for i in C_1_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if C_1_ratio_17_18[i][k]==[] and C_1_ratio_17_18[i][k-20]!=[] and C_1_ratio_17_18[i][k+20]!=[]:
            C_1_ratio_17_18[i][k]=[np.mean([np.mean(C_1_ratio_17_18[i][k-20]),np.mean(C_1_ratio_17_18[i][k+20])])]
            
for i in C_1_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if C_1_ratio_18_19[i][k]==[] and C_1_ratio_18_19[i][k-20]!=[] and C_1_ratio_18_19[i][k+20]!=[]:
            C_1_ratio_18_19[i][k]=[np.mean([np.mean(C_1_ratio_18_19[i][k-20]),np.mean(C_1_ratio_18_19[i][k+20])])]
            
C_1_ratio_8_9_key=list(C_1_ratio_8_9.keys())
for i in C_1_ratio_8_9_key:
    k=0
    if C_1_ratio_8_9[i][480]==[]:k+=1
    if C_1_ratio_8_9[i][500]==[]:k+=1
    if C_1_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del C_1_ratio_8_9[i]
        
C_1_ratio_9_10_key=list(C_1_ratio_9_10.keys())
for i in C_1_ratio_9_10_key:
    k=0
    if C_1_ratio_9_10[i][540]==[]:k+=1
    if C_1_ratio_9_10[i][560]==[]:k+=1
    if C_1_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del C_1_ratio_9_10[i]

C_1_ratio_17_18_key=list(C_1_ratio_17_18.keys())
for i in C_1_ratio_17_18_key:
    k=0
    if C_1_ratio_17_18[i][17*60]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_1_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del C_1_ratio_17_18[i]

C_1_ratio_18_19_key=list(C_1_ratio_18_19.keys())
for i in  C_1_ratio_18_19_key:
    k=0
    if C_1_ratio_18_19[i][18*60]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_1_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del C_1_ratio_18_19[i]
        
for i in C_1_ratio_8_9:
    del C_1_ratio_8_9[i][8*60-20]
    del C_1_ratio_8_9[i][9*60]
    
for i in C_1_ratio_9_10:
    del C_1_ratio_9_10[i][9*60-20]
    del C_1_ratio_9_10[i][10*60]
    
for i in C_1_ratio_17_18:
    del C_1_ratio_17_18[i][17*60-20]
    del C_1_ratio_17_18[i][18*60]
    
for i in C_1_ratio_18_19:
    del C_1_ratio_18_19[i][18*60-20]
    del C_1_ratio_18_19[i][19*60]
    
for i in C_1_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in C_1_ratio_8_9[i]: 
        if C_1_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_8_9[i][j]))
            temp2.append(np.mean(C_1_ratio_8_9[i][j]))
        if C_1_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in C_1_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in C_1_ratio_9_10[i]: 
        if C_1_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_9_10[i][j]))
            temp2.append(np.mean(C_1_ratio_9_10[i][j]))
        if C_1_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in C_1_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in C_1_ratio_17_18[i]: 
        if C_1_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_17_18[i][j]))
            temp2.append(np.mean(C_1_ratio_17_18[i][j]))
        if C_1_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in C_1_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in C_1_ratio_18_19[i]: 
        if C_1_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(C_1_ratio_18_19[i][j]))
            temp2.append(np.mean(C_1_ratio_18_19[i][j]))
        if C_1_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_1_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
C_1_ratio_8_9_480=[]
C_1_ratio_8_9_500=[]
C_1_ratio_8_9_520=[]

for i in C_1_ratio_8_9:
    for j in C_1_ratio_8_9[i]:
        if j==480: C_1_ratio_8_9_480.extend(C_1_ratio_8_9[i][j])
        if j==500: C_1_ratio_8_9_500.extend(C_1_ratio_8_9[i][j])
        if j==520: C_1_ratio_8_9_520.extend(C_1_ratio_8_9[i][j])
    
    
C_1_ratio_9_10_540=[]
C_1_ratio_9_10_560=[]
C_1_ratio_9_10_580=[]

for i in C_1_ratio_9_10:
    for j in C_1_ratio_9_10[i]:
        if j==540: C_1_ratio_9_10_540.extend(C_1_ratio_9_10[i][j])
        if j==560: C_1_ratio_9_10_560.extend(C_1_ratio_9_10[i][j])
        if j==580: C_1_ratio_9_10_580.extend(C_1_ratio_9_10[i][j])
            
C_1_ratio_17_18_1020=[]
C_1_ratio_17_18_1040=[]
C_1_ratio_17_18_1060=[]

for i in C_1_ratio_17_18:
    for j in C_1_ratio_17_18[i]:
        if j==1020: C_1_ratio_17_18_1020.extend(C_1_ratio_17_18[i][j])
        if j==1040: C_1_ratio_17_18_1040.extend(C_1_ratio_17_18[i][j])
        if j==1060: C_1_ratio_17_18_1060.extend(C_1_ratio_17_18[i][j])
    
C_1_ratio_18_19_1080=[]
C_1_ratio_18_19_1100=[]
C_1_ratio_18_19_1120=[]

for i in C_1_ratio_18_19:
    for j in C_1_ratio_18_19[i]:
        if j==1080: C_1_ratio_18_19_1080.extend(C_1_ratio_18_19[i][j])
        if j==1100: C_1_ratio_18_19_1100.extend(C_1_ratio_18_19[i][j])
        if j==1120: C_1_ratio_18_19_1120.extend(C_1_ratio_18_19[i][j])
            
C_1_ratio_8_9_480.sort()
C_1_ratio_8_9_480_used=C_1_ratio_8_9_480[5:len(C_1_ratio_8_9_480)-5]

C_1_ratio_8_9_500.sort()
C_1_ratio_8_9_500_used=C_1_ratio_8_9_500[5:len(C_1_ratio_8_9_500)-5]

C_1_ratio_8_9_520.sort()
C_1_ratio_8_9_520_used=C_1_ratio_8_9_520[5:len(C_1_ratio_8_9_520)-5]

C_1_ratio_9_10_540.sort()
C_1_ratio_9_10_540_used=C_1_ratio_9_10_540[5:len(C_1_ratio_9_10_540)-5]

C_1_ratio_9_10_560.sort()
C_1_ratio_9_10_560_used=C_1_ratio_9_10_560[5:len(C_1_ratio_9_10_560)-5]

C_1_ratio_9_10_580.sort()
C_1_ratio_9_10_580_used=C_1_ratio_9_10_580[5:len(C_1_ratio_9_10_580)-5]

C_1_ratio_17_18_1020.sort()
C_1_ratio_17_18_1020_used=C_1_ratio_17_18_1020[5:len(C_1_ratio_17_18_1020)-5]

C_1_ratio_17_18_1040.sort()
C_1_ratio_17_18_1040_used=C_1_ratio_17_18_1040[5:len(C_1_ratio_17_18_1040)-5]

C_1_ratio_17_18_1060.sort()
C_1_ratio_17_18_1060_used=C_1_ratio_17_18_1060[5:len(C_1_ratio_17_18_1060)-5]

C_1_ratio_18_19_1080.sort()
C_1_ratio_18_19_1080_used=C_1_ratio_18_19_1080[5:len(C_1_ratio_18_19_1080)-5]

C_1_ratio_18_19_1100.sort()
C_1_ratio_18_19_1100_used=C_1_ratio_18_19_1100[5:len(C_1_ratio_18_19_1100)-5]

C_1_ratio_18_19_1120.sort()
C_1_ratio_18_19_1120_used=C_1_ratio_18_19_1120[5:len(C_1_ratio_18_19_1120)-5]

names=locals()
C_1_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    C_1_ratio.append(np.mean(names['C_1_ratio_'+str(i)+'_used']))
    
C_1_ratio_8=C_1_ratio[0:3]
C_1_ratio_9=C_1_ratio[3:6]
C_1_ratio_17=C_1_ratio[6:9]
C_1_ratio_18=C_1_ratio[9:12]



In [240]:
print(C_1_ratio_8)
print(C_1_ratio_9)
print(C_1_ratio_17)
print(C_1_ratio_18)

[1.0085446631871167, 1.0135826326873116, 0.9693400182600777]
[0.9649815049634551, 1.0068986583841009, 1.0131470826840883]
[0.9455038549039972, 0.9966929710379031, 1.0406337231620946]
[0.95167652707838, 1.0515667415053014, 0.9895062988612441]


In [252]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_train/C_1.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_test/C_1.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_p5= pd.read_csv('/Users/vayne/Desktop/dm_pro_engin/used_data_train/5min_shifted_C_1.csv')
train_m5= pd.read_csv('/Users/vayne/Desktop/dm_pro_engin/used_data_train/-5min_shifted_C_1.csv')

train_p5.reset_index(drop=True, inplace=True)
train_m5.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)
#train1 = pd.concat([train,train_p5,train_m5])
train1 = pd.concat([train,train_p5])
train1.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train1)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=30, learning_rate=0.1, 
                                 max_depth=4, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)

test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in C_1_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in C_1_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in C_1_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in C_1_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])
    

final_result.sort(key = lambda x: x[0]) 

0.1090407309927677
0.1936652854065166
          label           0
0    140.245000  166.330834
1    231.923333  197.703157
2    146.070000  162.850243
3    190.573333  159.720588
4    207.370000  201.276029
..          ...         ...
543  283.800000  231.149707
544  230.510000  239.951900
545  198.123333  227.890675
546  261.010000  239.891815
547  267.110000  235.434175

[548 rows x 2 columns]
         label           0
0   158.785000  178.241258
1   162.683333  173.201385
2   193.376667  187.668158
3   170.245000  161.509270
4   171.540000  165.759641
5   166.753333  169.378819
6   158.060000  170.919259
7   146.846667  180.816640
8   210.020000  175.946243
9   157.720000  201.735223
10  168.473333  168.222627
11  187.665000  171.920564
12  270.883333  173.739059
13  171.320000  170.282751
14  280.493333  187.773710
15  207.370000  177.714726
16  269.365000  188.100980
17  404.243333  185.924575
18  266.843333  193.268456
19  226.350000  186.674408
20  168.280000  180.640762
21  277.

In [253]:
real_value_df_C_1.loc[real_value_df_C_1['route'] == 'C_1']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_C_1_time=list(real_value_df_C_1['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_C_1_time:final_result3.append(i[0])
        

In [254]:
pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    

#load the real value 
real_value_df_C_1=real_value_df.loc[real_value_df['route'] =='C_1']

real_value=list(real_value_df_C_1['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))


mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)

0.21765305341722707

In [255]:
mape_list

[0.33643795718510394,
 0.013098199262674852,
 0.10419386044657229,
 0.39480604218711984,
 0.20530200524205394,
 0.5746915323520349,
 0.008637763338186957,
 0.18066310940633099,
 0.01661950372560286,
 0.26246832812221366,
 0.5313404273170339,
 0.104319967047365,
 0.07281126184630732,
 0.19469887989798249,
 0.06809807748569491,
 0.2918980132689792,
 0.19306766175949924,
 0.2800772994712037,
 0.06912435337207512,
 0.06800414879341846,
 0.32200955358856037,
 0.32442921639237937,
 0.055671707772592866,
 0.06302216424136277,
 0.09387200589411822,
 0.40526065980035425,
 0.11265844379738346,
 0.34403533089960403,
 0.21472005535527616,
 0.3995871103858297,
 0.08391626594034306,
 0.40280689530772934,
 0.18770468783812344,
 0.10726199667793068,
 0.15346472692296637,
 0.08271676210411893,
 0.07024295848520642,
 0.01659301727585674,
 0.3377570067996331,
 0.6016301197076681,
 0.5986820790132047,
 0.6250400611070762,
 0.5880919036931207,
 0.401819422939429,
 0.00919329127746074,
 0.07395854013313669,

In [197]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

[[134.51, 181.13737694342694],
 [183.06, 182.0422100298492],
 [158.02, 175.56278175432],
 [130.53, 183.18893004825014],
 [151.99, 184.3257298169966],
 [417.44, 182.37903724409978],
 [185.55, 192.25294909486223],
 [238.49, 200.72871788906284],
 [179.15, 166.14209245815],
 [263.94, 183.58075861074613],
 [390.85, 172.74635058828844],
 [158.18, 177.975013794499],
 [189.34, 178.86404997110222],
 [140.53, 171.05668139298822],
 [158.96, 167.67144518971315],
 [250.19, 174.95480726051755],
 [220.91, 176.040509242523],
 [233.4, 156.63679315659934],
 [190.28, 165.11703250639013],
 [198.43, 172.3964724218638],
 [283.97, 196.5067592487531],
 [314.9, 217.13257249430845],
 [200.43, 214.99274237339375],
 [178.94, 216.06669270834237],
 [200.76, 206.63543859271283],
 [138.53, 215.097016196131],
 [182.56, 224.44046431595564],
 [152.07, 225.83325517872518],
 [226.48, 180.35052273175356],
 [312.25, 190.11461179923148],
 [174.53, 185.08915098217793],
 [149.01, 204.5165451162605],
 [165.61, 192.4465672280482

In [198]:
for i in pred_value:
    print(i)

181.13737694342694
182.0422100298492
175.56278175432
183.18893004825014
184.3257298169966
182.37903724409978
192.25294909486223
200.72871788906284
166.14209245815
183.58075861074613
172.74635058828844
177.975013794499
178.86404997110222
171.05668139298822
167.67144518971315
174.95480726051755
176.040509242523
156.63679315659934
165.11703250639013
172.3964724218638
196.5067592487531
217.13257249430845
214.99274237339375
216.06669270834237
206.63543859271283
215.097016196131
224.44046431595564
225.83325517872518
180.35052273175356
190.11461179923148
185.08915098217793
204.5165451162605
192.4465672280482
159.00034867932482
159.79460096819759
157.2937804246077
164.12635440893638
165.14485919352208
178.43537008773257
188.09577372801687
196.3882670124673
176.9300614454557
195.50105828469933
183.96314848245314
175.04936697068834
167.40850829882834
173.868814008232
181.42137922773304
191.43263503198955
201.79670423768002
210.6932242474007
195.5976704972794
216.12806364185084
203.37280734854536

## C3


In [63]:

#find the travel time at each time window
file_path='/Users/vayne/Desktop/dm_pro_engin/phase1_training/table5.csv'

# Step 1: Load trajectories
fr = open(file_path, 'r')
fr.readline()  # skip the header
traj_data = fr.readlines()
fr.close()

# 建立各Link時間的dictionary
for i in range(24):
    link_time[str(i+100)]={}
# Step 2: Create a dictionary to store travel time for each route per time window
travel_times = {}  # key: route_id. Value is also a dictionary of which key is the start time for the time window and value is a list of travel times
for i in range(len(traj_data)):
    each_traj = traj_data[i].replace('"', '').split(',')
    intersection_id = each_traj[0]
    tollgate_id = each_traj[1]

    route_id = intersection_id + '_' + tollgate_id
    if route_id not in travel_times.keys():
        travel_times[route_id] = {}

    trace_start_time = each_traj[3]
    travel_seq = each_traj[4]
    trace_start_time = datetime.strptime(trace_start_time, "%Y-%m-%d %H:%M:%S")
    time_window_minute = math.floor(trace_start_time.minute / 20) * 20
    start_time_window = datetime(trace_start_time.year, trace_start_time.month, trace_start_time.day,
                                     trace_start_time.hour, time_window_minute, 0)
    time=start_time_window
    tt = float(each_traj[-1])
    # 國慶不管，因道路狀態不一樣
    if(time.month==10 and time.day in [1,2,3,4,5,6,7]):
        continue
    # 中秋節不管，因道路狀態不一樣
    if(time.month==9 and time.day in [15,16,17]):
        continue
    if start_time_window not in travel_times[route_id].keys():
        travel_times[route_id][start_time_window] = [tt]
    else:
        travel_times[route_id][start_time_window].append(tt)
    
C_3_travel_times=travel_times['C_3']
C_3_travel_times_need={}
for i in C_3_travel_times:
    if i.hour in [7,8,9,16,17,18]:
        if i not in C_3_travel_times_need:
            C_3_travel_times_need[i]=[]
        if i in C_3_travel_times_need:
            C_3_travel_times_need[i].extend(C_3_travel_times[i])
        
C_3_ratio_8_9={}
C_3_ratio_9_10={}
C_3_ratio_17_18={}
C_3_ratio_18_19={}

date_key=[236, 237, 238, 239, 240, 241, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327]

for i in date_key:
    C_3_ratio_8_9[i]={}
    C_3_ratio_9_10[i]={}
    C_3_ratio_17_18[i]={}
    C_3_ratio_18_19[i]={}
    
    for j in [7*60+40,8*60+0,8*60+20,8*60+40,9*60]:
        C_3_ratio_8_9[i][j]=[]
    
    for j in [8*60+40,9*60,9*60+20,9*60+40,10*60]:
        C_3_ratio_9_10[i][j]=[]
        
    for j in [16*60+40,17*60+0,17*60+20,17*60+40,18*60+0]:
        C_3_ratio_17_18[i][j]=[]
        
    for j in [17*60+40,18*60+0,18*60+20,18*60+40,19*60+0]:
        C_3_ratio_18_19[i][j]=[]

for i in C_3_travel_times_need:
    temp_date=i.month*31+i.day
    temp_hour=i.hour
    temp_min=i.minute
    
    
    if temp_hour*60+temp_min>=460 and temp_hour*60+temp_min<=540:
        C_3_ratio_8_9[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=520 and temp_hour*60+temp_min<=600:
        C_3_ratio_9_10[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
    
    if temp_hour*60+temp_min>=16*60+40 and temp_hour*60+temp_min<=18*60:
        C_3_ratio_17_18[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
        
    if temp_hour*60+temp_min>=17*60+40 and temp_hour*60+temp_min<=19*60:
        C_3_ratio_18_19[temp_date][int(temp_hour*60+temp_min)].append(C_3_travel_times_need[i])
        
for i in date_key:
    k=0
    if C_3_ratio_8_9[i][480]==[]:k+=1
    if C_3_ratio_8_9[i][500]==[]:k+=1
    if C_3_ratio_8_9[i][520]==[]:k+=1 
    if k>1:del C_3_ratio_8_9[i]
        
for i in date_key:
    k=0
    if C_3_ratio_9_10[i][540]==[]:k+=1
    if C_3_ratio_9_10[i][560]==[]:k+=1
    if C_3_ratio_9_10[i][580]==[]:k+=1 
    if k>1:del C_3_ratio_9_10[i]
        
for i in date_key:
    k=0
    if C_3_ratio_17_18[i][17*60]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>1:del C_3_ratio_17_18[i]
        
for i in date_key:
    k=0
    if C_3_ratio_18_19[i][18*60]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>1:del C_3_ratio_18_19[i]
        
for i in C_3_ratio_8_9:
    for k in [480,500,520]:
        if C_3_ratio_8_9[i][k]==[] and C_3_ratio_8_9[i][k-20]!=[] and C_3_ratio_8_9[i][k+20]!=[]:
            C_3_ratio_8_9[i][k]=[np.mean([np.mean(C_3_ratio_8_9[i][k-20]),np.mean(C_3_ratio_8_9[i][k+20])])]
            
for i in C_3_ratio_9_10:
    for k in [9*60,9*60+20,9*60+40]:
        if C_3_ratio_9_10[i][k]==[] and C_3_ratio_9_10[i][k-20]!=[] and C_3_ratio_9_10[i][k+20]!=[]:
            C_3_ratio_9_10[i][k]=[np.mean([np.mean(C_3_ratio_9_10[i][k-20]),np.mean(C_3_ratio_9_10[i][k+20])])]
            
for i in C_3_ratio_17_18:
    for k in [17*60,17*60+20,17*60+40]:
        if C_3_ratio_17_18[i][k]==[] and C_3_ratio_17_18[i][k-20]!=[] and C_3_ratio_17_18[i][k+20]!=[]:
            C_3_ratio_17_18[i][k]=[np.mean([np.mean(C_3_ratio_17_18[i][k-20]),np.mean(C_3_ratio_17_18[i][k+20])])]
            
for i in C_3_ratio_18_19:
    for k in [18*60,18*60+20,18*60+40]:
        if C_3_ratio_18_19[i][k]==[] and C_3_ratio_18_19[i][k-20]!=[] and C_3_ratio_18_19[i][k+20]!=[]:
            C_3_ratio_18_19[i][k]=[np.mean([np.mean(C_3_ratio_18_19[i][k-20]),np.mean(C_3_ratio_18_19[i][k+20])])]
            
C_3_ratio_8_9_key=list(C_3_ratio_8_9.keys())
for i in C_3_ratio_8_9_key:
    k=0
    if C_3_ratio_8_9[i][480]==[]:k+=1
    if C_3_ratio_8_9[i][500]==[]:k+=1
    if C_3_ratio_8_9[i][520]==[]:k+=1 
    if k>0:del C_3_ratio_8_9[i]
        
C_3_ratio_9_10_key=list(C_3_ratio_9_10.keys())
for i in C_3_ratio_9_10_key:
    k=0
    if C_3_ratio_9_10[i][540]==[]:k+=1
    if C_3_ratio_9_10[i][560]==[]:k+=1
    if C_3_ratio_9_10[i][580]==[]:k+=1 
    if k>0:del C_3_ratio_9_10[i]

C_3_ratio_17_18_key=list(C_3_ratio_17_18.keys())
for i in C_3_ratio_17_18_key:
    k=0
    if C_3_ratio_17_18[i][17*60]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+20]==[]:k+=1 
    if C_3_ratio_17_18[i][17*60+40]==[]:k+=1
    if k>0:del C_3_ratio_17_18[i]

C_3_ratio_18_19_key=list(C_3_ratio_18_19.keys())
for i in  C_3_ratio_18_19_key:
    k=0
    if C_3_ratio_18_19[i][18*60]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+20]==[]:k+=1 
    if C_3_ratio_18_19[i][18*60+40]==[]:k+=1
    if k>0:del C_3_ratio_18_19[i]
        
for i in C_3_ratio_8_9:
    del C_3_ratio_8_9[i][8*60-20]
    del C_3_ratio_8_9[i][9*60]
    
for i in C_3_ratio_9_10:
    del C_3_ratio_9_10[i][9*60-20]
    del C_3_ratio_9_10[i][10*60]
    
for i in C_3_ratio_17_18:
    del C_3_ratio_17_18[i][17*60-20]
    del C_3_ratio_17_18[i][18*60]
    
for i in C_3_ratio_18_19:
    del C_3_ratio_18_19[i][18*60-20]
    del C_3_ratio_18_19[i][19*60]
    
for i in C_3_ratio_8_9:
    temp={480:[],500:[],520:[]}
    temp2=[]
    for j in C_3_ratio_8_9[i]: 
        if C_3_ratio_8_9[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_8_9[i][j]))
            temp2.append(np.mean(C_3_ratio_8_9[i][j]))
        if C_3_ratio_8_9[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_8_9[i][k]=temp[k]/np.mean(temp2)
        
for i in C_3_ratio_9_10:
    temp={540:[],560:[],580:[]}
    temp2=[]
    for j in C_3_ratio_9_10[i]: 
        if C_3_ratio_9_10[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_9_10[i][j]))
            temp2.append(np.mean(C_3_ratio_9_10[i][j]))
        if C_3_ratio_9_10[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_9_10[i][k]=temp[k]/np.mean(temp2)
        
for i in C_3_ratio_17_18:
    temp={17*60:[],17*60+20:[],17*60+40:[]}
    temp2=[]
    for j in C_3_ratio_17_18[i]: 
        if C_3_ratio_17_18[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_17_18[i][j]))
            temp2.append(np.mean(C_3_ratio_17_18[i][j]))
        if C_3_ratio_17_18[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_17_18[i][k]=temp[k]/np.mean(temp2)
        
for i in C_3_ratio_18_19:
    temp={18*60:[],18*60+20:[],18*60+40:[]}
    temp2=[]
    for j in C_3_ratio_18_19[i]: 
        if C_3_ratio_18_19[i][j]!=[]:
            temp[j].append(np.mean(C_3_ratio_18_19[i][j]))
            temp2.append(np.mean(C_3_ratio_18_19[i][j]))
        if C_3_ratio_18_19[i][j]==[]:
            temp[j].append(0)
            temp2.append(0)
    for k in temp:
        C_3_ratio_18_19[i][k]=temp[k]/np.mean(temp2)
        
C_3_ratio_8_9_480=[]
C_3_ratio_8_9_500=[]
C_3_ratio_8_9_520=[]

for i in C_3_ratio_8_9:
    for j in C_3_ratio_8_9[i]:
        if j==480: C_3_ratio_8_9_480.extend(C_3_ratio_8_9[i][j])
        if j==500: C_3_ratio_8_9_500.extend(C_3_ratio_8_9[i][j])
        if j==520: C_3_ratio_8_9_520.extend(C_3_ratio_8_9[i][j])
    
    
C_3_ratio_9_10_540=[]
C_3_ratio_9_10_560=[]
C_3_ratio_9_10_580=[]

for i in C_3_ratio_9_10:
    for j in C_3_ratio_9_10[i]:
        if j==540: C_3_ratio_9_10_540.extend(C_3_ratio_9_10[i][j])
        if j==560: C_3_ratio_9_10_560.extend(C_3_ratio_9_10[i][j])
        if j==580: C_3_ratio_9_10_580.extend(C_3_ratio_9_10[i][j])
            
C_3_ratio_17_18_1020=[]
C_3_ratio_17_18_1040=[]
C_3_ratio_17_18_1060=[]

for i in C_3_ratio_17_18:
    for j in C_3_ratio_17_18[i]:
        if j==1020: C_3_ratio_17_18_1020.extend(C_3_ratio_17_18[i][j])
        if j==1040: C_3_ratio_17_18_1040.extend(C_3_ratio_17_18[i][j])
        if j==1060: C_3_ratio_17_18_1060.extend(C_3_ratio_17_18[i][j])
    
C_3_ratio_18_19_1080=[]
C_3_ratio_18_19_1100=[]
C_3_ratio_18_19_1120=[]

for i in C_3_ratio_18_19:
    for j in C_3_ratio_18_19[i]:
        if j==1080: C_3_ratio_18_19_1080.extend(C_3_ratio_18_19[i][j])
        if j==1100: C_3_ratio_18_19_1100.extend(C_3_ratio_18_19[i][j])
        if j==1120: C_3_ratio_18_19_1120.extend(C_3_ratio_18_19[i][j])
            
C_3_ratio_8_9_480.sort()
C_3_ratio_8_9_480_used=C_3_ratio_8_9_480[5:len(C_3_ratio_8_9_480)-5]

C_3_ratio_8_9_500.sort()
C_3_ratio_8_9_500_used=C_3_ratio_8_9_500[5:len(C_3_ratio_8_9_500)-5]

C_3_ratio_8_9_520.sort()
C_3_ratio_8_9_520_used=C_3_ratio_8_9_520[5:len(C_3_ratio_8_9_520)-5]

C_3_ratio_9_10_540.sort()
C_3_ratio_9_10_540_used=C_3_ratio_9_10_540[5:len(C_3_ratio_9_10_540)-5]

C_3_ratio_9_10_560.sort()
C_3_ratio_9_10_560_used=C_3_ratio_9_10_560[5:len(C_3_ratio_9_10_560)-5]

C_3_ratio_9_10_580.sort()
C_3_ratio_9_10_580_used=C_3_ratio_9_10_580[5:len(C_3_ratio_9_10_580)-5]

C_3_ratio_17_18_1020.sort()
C_3_ratio_17_18_1020_used=C_3_ratio_17_18_1020[5:len(C_3_ratio_17_18_1020)-5]

C_3_ratio_17_18_1040.sort()
C_3_ratio_17_18_1040_used=C_3_ratio_17_18_1040[5:len(C_3_ratio_17_18_1040)-5]

C_3_ratio_17_18_1060.sort()
C_3_ratio_17_18_1060_used=C_3_ratio_17_18_1060[5:len(C_3_ratio_17_18_1060)-5]

C_3_ratio_18_19_1080.sort()
C_3_ratio_18_19_1080_used=C_3_ratio_18_19_1080[5:len(C_3_ratio_18_19_1080)-5]

C_3_ratio_18_19_1100.sort()
C_3_ratio_18_19_1100_used=C_3_ratio_18_19_1100[5:len(C_3_ratio_18_19_1100)-5]

C_3_ratio_18_19_1120.sort()
C_3_ratio_18_19_1120_used=C_3_ratio_18_19_1120[5:len(C_3_ratio_18_19_1120)-5]

names=locals()
C_3_ratio=[]
key1=['8_9_480','8_9_500','8_9_520','9_10_540','9_10_560','9_10_580','17_18_1020','17_18_1040','17_18_1060','18_19_1080','18_19_1100','18_19_1120']
for i in key1:
    C_3_ratio.append(np.mean(names['C_3_ratio_'+str(i)+'_used']))
    
C_3_ratio_8=C_3_ratio[0:3]
C_3_ratio_9=C_3_ratio[3:6]
C_3_ratio_17=C_3_ratio[6:9]
C_3_ratio_18=C_3_ratio[9:12]



In [64]:
print(C_3_ratio_8)
print(C_3_ratio_9)
print(C_3_ratio_17)
print(C_3_ratio_18)

[0.9148328918283646, 1.0075582153926481, 1.0225619584493677]
[1.040282893182057, 0.9801969562756571, 0.9588049191875893]
[0.9750481330349163, 0.9561918653994332, 1.064546493530613]
[1.0017435848975271, 0.980500959518688, 0.9999905668560394]


In [103]:
train_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_train/C_3.csv'
test_path = '/Users/vayne/Desktop/dm_pro_engin/used_data_test/C_3.csv'

train = pd.read_csv(train_path)
test = pd.read_csv(test_path)

train_p5= pd.read_csv('/Users/vayne/Desktop/dm_pro_engin/used_data_train/5min_shifted_C_3.csv')
train_m5= pd.read_csv('/Users/vayne/Desktop/dm_pro_engin/used_data_train/-5min_shifted_C_3.csv')

train_p5.reset_index(drop=True, inplace=True)
train_m5.reset_index(drop=True, inplace=True)
train.reset_index(drop=True, inplace=True)
#train1 = pd.concat([train,train_p5,train_m5])
train1 = pd.concat([train,train_p5])
train1.reset_index(drop=True, inplace=True)

train_x, train_y = get_x_y(train1)
test_x, test_y = get_x_y(test)

gdbt = GradientBoostingRegressor(n_estimators=20, learning_rate=0.1, 
                                 max_depth=6, random_state=0, loss='ls').fit(train_x, train_y)

print(mape_error(train_y, gdbt.predict(train_x)))
print(mape_error(test_y, gdbt.predict(test_x)))
result = gdbt.predict(train_x)
train_compare = pd.concat([train_y, pd.Series(result)],axis = 1)
print(train_compare)

result = gdbt.predict(test_x)
test_compare = pd.concat([test_y, pd.Series(result)],axis = 1)
print(test_compare)


test_hr=list(test['hr'])
test_date=list(test['date'])

result_with_hrdate=[]
for i in range(len(result)):
    result_with_hrdate.append([test_date[i],test_hr[i],result[i]])
    
predict_result=[]
for i in result_with_hrdate:
    if i[1]==8:
        temp=[]
        for k in C_3_ratio_8:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==9:
        temp=[]
        for k in C_3_ratio_9:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==17:
        temp=[]
        for k in C_3_ratio_17:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
    if i[1]==18:
        temp=[]
        for k in C_3_ratio_18:
            temp.append([i[0],i[1],float(i[2]*k)])
        predict_result.extend(temp)
        
final_result=[]
for i in predict_result:
    final_result.append([i[0],i[1],i[2]])
    

final_result.sort(key = lambda x: x[0]) 

0.11766762204076897
0.15351527958288697
          label           0
0    132.050000  157.763895
1    219.150000  193.637728
2    166.150000  181.585406
3    161.786667  193.295409
4    207.950000  197.318651
..          ...         ...
506  242.460000  188.023270
507  152.910000  182.797675
508  250.595000  200.063809
509  138.943333  177.389491
510  286.130000  209.257904

[511 rows x 2 columns]
         label           0
0   194.430000  180.480309
1   188.860000  174.106268
2   168.170000  200.943538
3   169.465000  165.570336
4   196.430000  192.525483
5          NaN  166.034788
6   141.630000  193.873634
7   179.806667  179.922049
8   173.563333  177.072852
9   178.855000  188.882611
10  131.273333  170.457847
11  147.816667  197.004775
12  152.610000  171.660899
13  155.760000  192.196575
14  189.830000  184.986649
15  147.035000  182.912230
16  151.936667  183.495366
17  224.380000  185.657256
18  204.046667  204.403979
19  199.775000  195.867465
20  146.396667  172.041932
21  14

In [104]:
#有的时间窗口真实数据里是没有值的，所以要筛一下，去掉predicted value里的一些值
real_value_df_C_3.loc[real_value_df_C_3['route'] == 'C_3']

result_index=[]
for i in range(0,84,3):
    result_index.append([i,i+3])

final_result2=[]
for i in result_index:
    temp_result=final_result[i[0]:i[1]]
    temp_date=datetime.strptime(temp_result[0][0], "%m-%d")
    temp_month=temp_date.month
    temp_day=temp_date.day
    temp_hr=temp_result[0][1]
    
    for j in range(len(temp_result)):
        final_result2.append([temp_result[j],datetime.strptime('2016-'+str(temp_month)+'-'+str(temp_day)+' '+str(temp_hr)+':'+str(20*j)+':00', "%Y-%m-%d %H:%M:%S")])
        
real_value_df_C_3_time=list(real_value_df_C_3['time'])

final_result3=[]
for i in final_result2:
    if i[1] in real_value_df_C_3_time:final_result3.append(i[0])

pred_value=[]
for i in final_result3:
    pred_value.append(i[2])
    

#load the real value 
real_value_df_C_3=real_value_df.loc[real_value_df['route'] =='C_3']

real_value=list(real_value_df_C_3['label'])

real_value1=[]
for i in real_value:
    real_value1.append(float(i))


mape_list=[]
for i in range(len(pred_value)):
    mape_list.append(np.abs((real_value1[i] - pred_value[i]) / real_value1[i]))
    
np.mean(mape_list)



0.23784763244182056

In [105]:
for i in mape_list:
    if i>1:
        mape_list.remove(i)

np.mean(mape_list)

0.2191361663511697

In [96]:

test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

[[233.55, 181.84441794837912],
 [155.31, 184.55229812990586],
 [216.72, 187.16982940587067],
 [178.4, 176.35904453747565],
 [144.3, 172.5101453979634],
 [189.83, 180.37088710680172],
 [84.47, 197.8138844123198],
 [204.89, 197.46771667692366],
 [188.63, 159.27814109008816],
 [175.91, 175.42220117058963],
 [202.04, 178.03444688760956],
 [209.15, 184.20585892413982],
 [143.38, 173.5662707028539],
 [168.16, 169.77832168266852],
 [108.94, 178.34822812182406],
 [185.13, 174.89918616395502],
 [273.95, 190.56691205035276],
 [123.48, 186.525816521199],
 [170.41, 190.23342627617725],
 [141.54, 202.46231233816974],
 [194.8, 205.47721754819517],
 [148.35, 196.49134905457228],
 [209.36, 185.1421604066338],
 [143.44, 178.9168138228537],
 [138.56, 175.45677609586403],
 [173.81, 195.3393482185847],
 [211.71, 184.4179672873229],
 [124.56, 166.82175248542237],
 [214.37, 169.30592726790633],
 [106.63, 177.3243817772643],
 [155.75, 167.08226236410732],
 [131.44, 163.43582178870312],
 [193.78, 177.52395756

In [106]:
test_compare=[]
for i in range(len(pred_value)):
    test_compare.append([real_value1[i],pred_value[i]])
test_compare

[[233.55, 181.84441794837912],
 [155.31, 184.55229812990586],
 [216.72, 187.16982940587067],
 [178.4, 176.35904453747565],
 [144.3, 172.5101453979634],
 [189.83, 180.37088710680172],
 [84.47, 197.8138844123198],
 [204.89, 197.46771667692366],
 [188.63, 159.27814109008816],
 [175.91, 175.42220117058963],
 [202.04, 178.03444688760956],
 [209.15, 184.20585892413982],
 [143.38, 173.5662707028539],
 [168.16, 169.77832168266852],
 [108.94, 178.34822812182406],
 [185.13, 174.89918616395502],
 [273.95, 190.56691205035276],
 [123.48, 186.525816521199],
 [170.41, 190.23342627617725],
 [141.54, 202.46231233816974],
 [194.8, 205.47721754819517],
 [148.35, 196.49134905457228],
 [209.36, 185.1421604066338],
 [143.44, 178.9168138228537],
 [138.56, 175.45677609586403],
 [173.81, 195.3393482185847],
 [211.71, 184.4179672873229],
 [124.56, 166.82175248542237],
 [214.37, 169.30592726790633],
 [106.63, 177.3243817772643],
 [155.75, 167.08226236410732],
 [131.44, 163.43582178870312],
 [193.78, 177.52395756

In [110]:
sum2=0.16232729428949813+0.13194938082850158+0.21883943302247055+\
0.18741485382049425+0.21765305341722707+0.2191361663511697

In [111]:
sum2/6

0.1895533636215602